In [ ]:
# b3_iframe_selenium.py
from __future__ import annotations

import argparse
import csv
import logging
import re
import time
from io import StringIO
from pathlib import Path
from typing import Iterable, List, Optional, Set

import pandas as pd
import requests
from bs4 import BeautifulSoup
from requests.adapters import HTTPAdapter, Retry

# Selenium imports are optional; import when needed to avoid heavy dependency at module import
try:
    from selenium import webdriver
    from selenium.common.exceptions import (
        ElementClickInterceptedException,
        StaleElementReferenceException,
        TimeoutException,
    )
    from selenium.webdriver.chrome.service import Service
    from selenium.webdriver.common.by import By
    from selenium.webdriver.chrome.options import Options
    from selenium.webdriver.support.ui import WebDriverWait
    from selenium.webdriver.support import expected_conditions as EC
    from webdriver_manager.chrome import ChromeDriverManager
except Exception:  # pragma: no cover - selenium optional
    webdriver = None



#download_histories.py
import os
import logging
from datetime import datetime
from typing import List, Dict, Optional
import pandas as pd
import yfinance as yf
from tqdm import tqdm
import traceback

# analysis_historical_data.py
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# cvm_downloader.py
import requests
from zipfile import ZipFile
from __future__ import annotations
import argparse
import sys
from concurrent.futures import ThreadPoolExecutor, as_completed
from typing import Iterable, List, Tuple, Dict

import requests
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry

# cvm_parser.py
import gc
from pathlib import Path

# news_crawler_rss.py
import feedparser
import urllib.parse



---

# 🧠 Arquitetura de Coleta e Processamento — Projeto Aurum

```mermaid
flowchart LR
    A["Web Scraping na página da B3<br/><b>Ticker</b>"]

    %% Yahoo Finance
    A --> B["Web Scraping no Yahoo Finance<br/><b>Dados Históricos</b>"]
    B --> C["<b>Backtests</b>"]
    C --> D["<b>Relatório de Desempenho</b>"]

    %% Google News
    A --> E["Web Scraping na página do Google News<br/><b>Dados de Notícias</b>"]
    E --> F["<b>Modelo de Sentimento</b>"]
    F --> G["<b>Score de Sentimento</b>"]

    %% CVM
    A --> H["Baixar por uma URL gov.br<br/><b>Baixar as pastas CVM</b>"]
    H --> I["<b>Parser e Processador</b>"]
    I --> J["<b>Dados Fundamentalistas</b>"]

    %% União
    G --> K["<b>DataFrame Fundamentalistas e Score de Sentimento</b>"]
    J --> K



---

### b3_iframe_full_extractor.py

### 📘 Descrição
Script responsável por **extrair automaticamente todos os tickers listados na B3** (como o índice IBRX100), mesmo quando o conteúdo está dentro de **iframes com paginação dinâmica**.  
O código realiza a extração utilizando **Selenium + BeautifulSoup + Pandas**, com fallback inteligente caso o download direto da tabela falhe.

### ⚙️ Funcionalidades Principais
- Extração de tickers da página da B3 (`https://sistemaswebb3-listados.b3.com.br/indexPage/day/IBXX`)
- Normalização dos tickers (ex: `PETR4` → `PETR4.SA`)
- Identificação automática de colunas e cabeçalhos de código
- Suporte a paginação automática e opções de "itens por página"
- Modo **headless** (sem abrir navegador) ou **debug** (com logs detalhados)
- Geração de arquivo CSV (`tickers_ibrx100_full.csv`) com todos os tickers coletados



### 🧩 Diagrama (classe/entidade) — saída do CSV
```mermaid
classDiagram
  class tickers_csv {
    + string Ticker
    --
    Código de negociação normalizado do ativo na B3
    Formato: <nome do papel> + .SA (ex.: PETR4.SA)
    Lista sem duplicatas; normaliza para maiúsculas
  }


In [ ]:
import pandas as pd
import requests
import base64
import json
import logging
from pathlib import Path
import time
import urllib3

# --- Configuração ---
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# Diretório para salvar
OUTPUT_DIR = Path(".") 
OUTPUT_FILENAME_CSV = "tickers_ibrx100_full.csv"
OUTPUT_FILENAME_PARQUET = "tickers_ibrx100_full.parquet"

def fetch_ibrx100_from_b3_api() -> pd.DataFrame:
    """
    Consome diretamente a API JSON da B3 para obter a composição do IBRX-100.
    """
    logger.info("Iniciando requisição à API da B3 (IndexProxy)...")
    
    try:
        # IBXX é o código do IBRX-100
        params = {
            "language": "pt-br",
            "pageNumber": 1,
            "pageSize": 120, 
            "index": "IBXX", 
            "segment": "1"
        }
        
        params_json = json.dumps(params)
        params_b64 = base64.b64encode(params_json.encode("utf-8")).decode("utf-8")
        
        url = f"https://sistemaswebb3-listados.b3.com.br/indexProxy/indexCall/GetPortfolioDay/{params_b64}"
        
        headers = {
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
        }
        
        # Desativa warnings de SSL
        urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
        response = requests.get(url, headers=headers, timeout=15, verify=False)
        
        if response.status_code != 200:
            logger.error(f"Erro na requisição: Status {response.status_code}")
            return None
            
        data = response.json()
        results = data.get('results', [])
        
        if not results:
            logger.warning("JSON retornado pela B3 está vazio na chave 'results'.")
            return None
            
        logger.info(f"API retornou {len(results)} ativos.")
        
        # Criar DataFrame Bruto
        df = pd.DataFrame(results)
        
        # --- DEBUG: Imprimir colunas encontradas para ver o nome correto ---
        logger.info(f"Colunas encontradas no JSON: {df.columns.tolist()}")
        
        # --- LÓGICA DE CORREÇÃO: Identificar a coluna do Ticker ---
        # A B3 varia entre 'codNeg', 'cod', 'acronym', 'asset' dependendo do endpoint
        coluna_ticker = None
        possiveis_nomes = ['codNeg', 'cod', 'acronym', 'symbol', 'identifier']
        
        for col in possiveis_nomes:
            if col in df.columns:
                coluna_ticker = col
                logger.info(f"Coluna de ticker identificada como: '{col}'")
                break
        
        if not coluna_ticker:
            logger.error("Não foi possível identificar a coluna de Ticker no DataFrame.")
            logger.error(f"Colunas disponíveis: {df.columns.tolist()}")
            return None

        # Selecionar e renomear
        # Também tentamos garantir a coluna de participação ('part' ou 'participation')
        coluna_part = 'part' if 'part' in df.columns else None
        
        colunas_selecao = [coluna_ticker]
        if coluna_part:
            colunas_selecao.append(coluna_part)
            
        df_final = df[colunas_selecao].copy()
        
        # Renomear para o padrão do nosso sistema
        rename_map = {coluna_ticker: 'ticker'}
        if coluna_part:
            rename_map[coluna_part] = 'participacao'
            
        df_final = df_final.rename(columns=rename_map)
        
        # 6. Normalização para yfinance (.SA)
        logger.info("Normalizando tickers (adicionando .SA)...")
        # Remove espaços em branco que a B3 as vezes deixa
        df_final['ticker'] = df_final['ticker'].str.strip()
        df_final['Ticker_Yahoo'] = df_final['ticker'].apply(lambda x: f"{x}.SA")
        
        return df_final

    except Exception as e:
        logger.error(f"Falha crítica no extrator da API B3: {e}")
        import traceback
        logger.error(traceback.format_exc())
        return None

def save_data(df: pd.DataFrame):
    OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
    
    path_csv = OUTPUT_DIR / OUTPUT_FILENAME_CSV
    path_parquet = OUTPUT_DIR / OUTPUT_FILENAME_PARQUET
    
    df.to_csv(path_csv, index=False, encoding='utf-8-sig')
    df.to_parquet(path_parquet, index=False)
    
    logger.info(f"💾 Arquivos salvos:")
    logger.info(f"   -> {path_csv}")
    logger.info(f"   -> {path_parquet}")

if __name__ == "__main__":
    start_time = time.time()
    
    df_result = fetch_ibrx100_from_b3_api()
    
    if df_result is not None and not df_result.empty:
        print("\n--- Amostra do IBRX-100 (API B3) ---")
        print(df_result.head())
        save_data(df_result)
    else:
        logger.error("Não foi possível gerar a lista de tickers.")
        
    print(f"\nTempo total: {time.time() - start_time:.2f} segundos")

2025-12-09 14:25:02,100 - INFO - Iniciando requisição à API da B3 (IndexProxy)...
2025-12-09 14:25:03,040 - INFO - API retornou 97 ativos.
2025-12-09 14:25:03,052 - ERROR - Falha crítica no extrator da API B3: "['codNeg'] not in index"
2025-12-09 14:25:03,164 - ERROR - Traceback (most recent call last):
  File "C:\Users\kaike\AppData\Local\Temp\ipykernel_22100\1422043272.py", line 71, in fetch_ibrx100_from_b3_api
    df_final = df[['codNeg', 'part']].copy()
               ~~^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\kaike\projeto_aurum\venv_aurum\Lib\site-packages\pandas\core\frame.py", line 4119, in __getitem__
    indexer = self.columns._get_indexer_strict(key, "columns")[1]
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\kaike\projeto_aurum\venv_aurum\Lib\site-packages\pandas\core\indexes\base.py", line 6212, in _get_indexer_strict
    self._raise_if_missing(keyarr, indexer, axis_name)
  File "c:\Users\kaike\projeto_aurum\venv_aurum\Lib\site-packages\panda


Tempo total: 1.07 segundos



---
### 🗂️ Histórico — downloader & combinador (b3_hist_downloader.py)

**Resumo rápido**  
Script que baixa históricos de preços (via `yfinance`), salva por ticker em **parquet** (e opcionalmente CSV), e permite combinar todos os parquets em um único arquivo *long-format*. Feito para rodar em batch com retry/backoff, pular tickers já salvos e gerar um resumo final.


#### ✨ Principais responsabilidades
- Baixar históricos de preço por lotes (batch) com `yfinance`.  
- Salvar cada ticker em `data/historical/{TICKER}.parquet` e `{TICKER}.csv`.  
- Gerenciar retries (exponencial), fallback ticker-a-ticker e logs.  
- Gerar CSV de resumo por execução (`download_summary_YYYYMMDDTHHMMSSZ.csv`).  
- Recombinar todos os parquets em um único `all_histories.parquet` / `all_histories.csv`.


#### 🧩 Funções principais (one-liners)
- `ticker_exists_local(ticker)` → verifica existência de `{ticker}.parquet`.  
- `save_history_df(ticker, df, save_csv=True)` → salva parquet e CSV; garante coluna `date`.  
- `download_batch(batch, start, threads)` → tenta baixar um batch via `yfinance.download` com retries e fallback.  
- `download_all_histories(tickers, start, force, save_summary, save_csv_per_ticker)` → orquestra o download em batches, salva e retorna um `DataFrame` resumo.  
- `combine_all_to_single_parquet(out_path, out_csv, tickers)` → concatena todos os parquets em formato long e salva.


#### ⚙️ Parâmetros principais (valores padrão)
| Parâmetro | Valor padrão |
|---:|:---|
| `HIST_DIR` | `data/historical/` |
| `DEFAULT_START` | `"2011-01-01"` |
| `BATCH_SIZE` | `15` |
| `MAX_ATTEMPTS` | `4` |
| `SLEEP_BETWEEN_BATCHES` | `1` (seg) |
| `SLEEP_BETWEEN_TICKERS` | `0.2` (seg) |


#### 📂 Saídas geradas
- `data/historical/{TICKER}.parquet` — parquet por ticker.  
- `data/historical/{TICKER}.csv` —  CSV por ticker.  
- `data/historical/download_summary_{ts}.csv` — resumo da execução.  
- `data/historical/all_histories.parquet` & `all_histories.csv` — concat final (long format).

---

In [ ]:
# logging simples
logging.basicConfig(level=logging.INFO, format="%(asctime)s %(levelname)s %(message)s")

# diretório onde os históricos serão salvos (compatível com seu script anterior)
HIST_DIR = os.path.join("data", "historical")
os.makedirs(HIST_DIR, exist_ok=True)

# parâmetros de download
DEFAULT_START = "2011-01-01"
BATCH_SIZE = 15
MAX_ATTEMPTS = 4          
SLEEP_BETWEEN_BATCHES = 1  
SLEEP_BETWEEN_TICKERS = 0.2

def ticker_exists_local(ticker: str) -> bool:
    """Verifica se já existe parquet salvo para ticker (usa para pular downloads)"""
    path = os.path.join(HIST_DIR, f"{ticker}.parquet")
    return os.path.isfile(path)

def save_history_df(ticker: str, df: pd.DataFrame, save_csv: bool = True):
    """Salva DataFrame em parquet e opcionalmente em CSV. Garante coluna 'date' se índice for DatetimeIndex."""
    if df is None or df.empty:
        raise ValueError("DataFrame nulo ou vazio")
    df = df.copy()
    # garantir que a coluna de data exista como coluna
    if isinstance(df.index, pd.DatetimeIndex):
        df.index.name = "date"
        df = df.reset_index()
    # converter coluna date para string ISO ao salvar CSV (mantém compatibilidade)
    out_parquet = os.path.join(HIST_DIR, f"{ticker}.parquet")
    out_csv = os.path.join(HIST_DIR, f"{ticker}.csv")
    try:
        df.to_parquet(out_parquet, index=False)
        logging.info("Saved %s rows for %s -> %s", len(df), ticker, out_parquet)
    except Exception as e:
        logging.exception("Erro salvando parquet para %s: %s", ticker, e)
        raise
    if save_csv:
        try:
            # padronizar data para ISO antes de salvar CSV (se existir)
            if 'date' in df.columns:
                df['date'] = pd.to_datetime(df['date']).dt.strftime('%Y-%m-%d')
            df.to_csv(out_csv, index=False)
            logging.info("Saved CSV for %s -> %s", ticker, out_csv)
        except Exception as e:
            logging.exception("Erro salvando CSV para %s: %s", ticker, e)
            # não raise — parquet já salvo, apenas logamos o problema

def download_batch(batch: List[str], start: str = DEFAULT_START, threads: bool = True) -> Dict[str, Optional[pd.DataFrame]]:
    """
    Tenta baixar um batch de tickers via yfinance.download.
    Retorna dict ticker -> DataFrame or None (se falhou).
    """
    joined = " ".join(batch)
    attempt = 0
    last_exc = None
    while attempt < MAX_ATTEMPTS:
        try:
            logging.info("yfinance.download attempt %d for batch size %d", attempt+1, len(batch))
            data = yf.download(tickers=joined, start=start, progress=False, threads=threads, group_by='ticker', auto_adjust=False, actions=True)
            result = {}
            if isinstance(data, pd.DataFrame) and isinstance(data.columns, pd.MultiIndex):
                for ticker in batch:
                    if ticker in data.columns.get_level_values(0):
                        df_t = data[ticker].copy()
                        result[ticker] = df_t
                    else:
                        try:
                            single = yf.download(ticker, start=start, progress=False, actions=True)
                            result[ticker] = single if not single.empty else None
                        except Exception:
                            result[ticker] = None
            else:
                for ticker in batch:
                    try:
                        df_t = yf.download(ticker, start=start, progress=False, actions=True)
                        result[ticker] = df_t if not df_t.empty else None
                    except Exception:
                        result[ticker] = None
            return result
        except Exception as e:
            last_exc = e
            logging.warning("Erro no yfinance.download (attempt %d): %s", attempt+1, str(e))
            attempt += 1
            time.sleep(2 ** attempt)  # backoff exponencial
    logging.error("Todas tentativas falharam para batch (%s). Último erro: %s", joined, last_exc)
    # fallback: tentar baixar ticker a ticker
    result = {}
    for ticker in batch:
        try:
            df_t = yf.download(ticker, start=start, progress=False, actions=True)
            result[ticker] = df_t if not df_t.empty else None
        except Exception as e:
            logging.warning("Fallback individual falhou para %s: %s", ticker, e)
            result[ticker] = None
    return result

def download_all_histories(tickers: List[str], start: str = DEFAULT_START, force: bool = False, save_summary: bool = True, save_csv_per_ticker: bool = True):
    """
    Processo principal: recebe lista de tickers (strings), baixa históricos e salva parquet + csv por ticker.
    - force: se True, re-baixa mesmo que arquivo exista.
    - save_csv_per_ticker: se True salva um CSV para cada ticker (além do parquet).
    - retorna um DataFrame resumo com status por ticker.
    """
    os.makedirs(HIST_DIR, exist_ok=True)
    tickers = [t for t in tickers if isinstance(t, str) and t.strip()]
    tickers = list(dict.fromkeys(tickers))
    summary = []
    for i in range(0, len(tickers), BATCH_SIZE):
        batch = tickers[i:i+BATCH_SIZE]
        to_download = [t for t in batch if force or not ticker_exists_local(t)]
        if not to_download:
            logging.info("Batch %d: todos já existem localmente — pulando.", i//BATCH_SIZE+1)
            for t in batch:
                summary.append({
                    "ticker": t,
                    "status": "skipped_local",
                    "rows": None,
                    "saved_parquet": os.path.join(HIST_DIR, f"{t}.parquet") if ticker_exists_local(t) else None,
                    "saved_csv": os.path.join(HIST_DIR, f"{t}.csv") if os.path.exists(os.path.join(HIST_DIR, f"{t}.csv")) else None
                })
            continue

        logging.info("Processando batch %d/%d (download %d/%d)", i//BATCH_SIZE+1, (len(tickers)+BATCH_SIZE-1)//BATCH_SIZE, len(to_download), len(batch))
        results = download_batch(to_download, start=start)
        for t in batch:
            df_t = results.get(t) if t in results else None
            if df_t is None or (isinstance(df_t, pd.DataFrame) and df_t.empty):
                logging.warning("Nenhum dado para %s em batch; tentativa isolada...", t)
                try:
                    single = yf.download(t, start=start, progress=False, actions=True)
                    df_t = single if not single.empty else None
                except Exception:
                    df_t = None
            if df_t is None or df_t.empty:
                logging.error("Falha obtendo dados para %s", t)
                summary.append({"ticker": t, "status": "failed", "rows": 0, "saved_parquet": None, "saved_csv": None})
            else:
                try:
                    save_history_df(t, df_t, save_csv=save_csv_per_ticker)
                    summary.append({
                        "ticker": t,
                        "status": "ok",
                        "rows": len(df_t),
                        "saved_parquet": os.path.join(HIST_DIR, f"{t}.parquet"),
                        "saved_csv": os.path.join(HIST_DIR, f"{t}.csv") if save_csv_per_ticker else None
                    })
                except Exception as e:
                    logging.exception("Erro salvando para %s: %s", t, e)
                    summary.append({"ticker": t, "status": "save_error", "rows": len(df_t) if isinstance(df_t, pd.DataFrame) else None, "saved_parquet": None, "saved_csv": None})
            time.sleep(SLEEP_BETWEEN_TICKERS)
        time.sleep(SLEEP_BETWEEN_BATCHES)

    df_summary = pd.DataFrame(summary)
    if save_summary:
        ts = datetime.utcnow().strftime("%Y%m%dT%H%M%SZ")
        summary_path = os.path.join(HIST_DIR, f"download_summary_{ts}.csv")
        df_summary.to_csv(summary_path, index=False)
        logging.info("Resumo salvo em %s", summary_path)
    return df_summary

def combine_all_to_single_parquet(out_path: str = os.path.join(HIST_DIR, "all_histories.parquet"), out_csv: Optional[str] = os.path.join(HIST_DIR, "all_histories.csv"), tickers: Optional[List[str]] = None):
    """
    Lê todos os parquets em HIST_DIR (ou tickers list) e concatena em formato long:
    columns: ['ticker','date', 'Open','High','Low','Close','Adj Close','Volume', 'Dividends','Stock Splits']
    Salva em parquet e opcionalmente em csv.
    """
    files = []
    if tickers:
        files = [os.path.join(HIST_DIR, f"{t}.parquet") for t in tickers if os.path.exists(os.path.join(HIST_DIR, f"{t}.parquet"))]
    else:
        files = [os.path.join(HIST_DIR, f) for f in os.listdir(HIST_DIR) if f.endswith(".parquet")]
    dfs = []
    for f in files:
        try:
            df = pd.read_parquet(f)
            if 'date' in df.columns:
                df['date'] = pd.to_datetime(df['date'])
            fname = os.path.basename(f).replace(".parquet","")
            if 'ticker' not in df.columns:
                df.insert(0, 'ticker', fname)
            dfs.append(df)
        except Exception as e:
            logging.warning("Erro lendo %s: %s", f, e)
    if not dfs:
        raise RuntimeError("Nenhum parquet encontrado para combinar.")
    big = pd.concat(dfs, ignore_index=True, sort=False)
    big.to_parquet(out_path, index=False)
    logging.info("Combined saved to %s (rows=%d)", out_path, len(big))
    if out_csv:
        try:
            # converter date para formato iso ao salvar CSV
            if 'date' in big.columns:
                big['date'] = pd.to_datetime(big['date']).dt.strftime('%Y-%m-%d')
            big.to_csv(out_csv, index=False)
            logging.info("Combined CSV saved to %s", out_csv)
        except Exception as e:
            logging.exception("Erro salvando combined CSV: %s", e)
    return big

if __name__ == "__main__":
    # 1) carregue a lista de tickers a partir do arquivo que você já salvou
    tickers_file = os.path.join("tickers_ibrx100_full.csv")
    if os.path.exists(tickers_file):
        df = pd.read_csv(tickers_file)
        if 'Ticker' in df.columns:
            tickers = df['Ticker'].dropna().astype(str).tolist()
        else:
            tickers = df.iloc[:,0].dropna().astype(str).tolist()
    else:
        raise RuntimeError(f"Não encontrou {tickers_file}. Coloque seu CSV de tickers no mesmo diretório ou edite este script.")

    # 2) opção: validar/normalizar tickers (garantir sufixo .SA)
    def normalize(t):
        t = str(t).strip().upper()
        if not t.endswith(".SA"):
            t = t.replace(".SA","") + ".SA"
        return t
    tickers = [normalize(t) for t in tickers]
    print("Tickers a baixar:", len(tickers), tickers[:10])

    # 3) executar (force=True re-baixa mesmo se já existir)
    summary_df = download_all_histories(tickers, start=DEFAULT_START, force=False, save_summary=True, save_csv_per_ticker=True)
    print(summary_df.head(50))

    # 4) opcional: combinar tudo em um único parquet e CSV (pode ser grande)
    combined = combine_all_to_single_parquet()
    print("Combined rows:", len(combined))



---

### 📋 Descrição das colunas do dataset histórico

| Coluna         | Tipo        | Breve descrição |
|---------------:|:-----------:|:----------------|
| `date`         | `date`      | Data da observação no formato ISO (`YYYY-MM-DD`). Use como índice temporal para séries. |
| `Open`         | `float`     | Preço de abertura do pregão (primeira transação considerada naquele dia). |
| `High`         | `float`     | Preço máximo registrado durante o pregão. |
| `Low`          | `float`     | Preço mínimo registrado durante o pregão. |
| `Close`        | `float`     | Preço de fechamento (última transação do dia). Não considera ajustes por eventos corporativos. |
| `Adj Close`    | `float`     | Preço de fechamento **ajustado** por splits e dividendos — use para cálculo de retornos total/consistentes em séries históricas. |
| `Volume`       | `int`       | Quantidade de ações negociadas no dia (unidades). |
| `Dividends`    | `float`     | Valor do dividendo pago por ação na data (se houver). Geralmente em moeda local (ex.: BRL para B3). |
| `Stock Splits` | `float`     | Fator de desdobramento/agrupamento (ex.: `2.0` → split 2-por-1). Zero ou `0.0` quando não houve evento. |

**Notas rápidas**
- `Adj Close` é a coluna recomendada para backtests e cálculo de retornos contínuos (corrige preços para manter consistência após eventos corporativos).  
- Converta `date` para `datetime` e defina como índice para operações de série temporal (`df['date'] = pd.to_datetime(df['date']); df.set_index('date', inplace=True)`).  
- Verifique a unidade/moeda conforme a fonte (normalmente moeda local do mercado, ex.: BRL para B3).  

---

In [2]:
import pandas as pd
import os
import io

def print_header(title):
    """
    Função auxiliar para imprimir um cabeçalho formatado no log.
    """
    print("\n" + "=" * 70)
    print(f" {title.upper()} ")
    print("=" * 70)

def analyze_dataframe(file_path):
    """
    Carrega e analisa um DataFrame de histórico de ações.
    """
    
    print(f"--- Iniciando Análise do Arquivo: {file_path} ---")

    # --- 1. Verificação e Carregamento dos Dados ---
    
    if not os.path.exists(file_path):
        print_header("[ERRO] Arquivo não encontrado")
        print(f"O arquivo no caminho '{file_path}' não foi localizado.")
        print("Por favor, verifique se o caminho está correto e a pasta 'historical' existe.")
        print("=" * 70)
        return

    try:
        # Tenta carregar o CSV. 
        # A coluna 'date' é convertida para datetime no carregamento.
        df = pd.read_csv(file_path, parse_dates=['date'])
        print(f"\n[SUCESSO] Arquivo carregado. Total de {len(df)} linhas e {len(df.columns)} colunas.")
    except Exception as e:
        print_header("[ERRO] Falha ao carregar o arquivo")
        print(f"Ocorreu um erro ao tentar ler o arquivo CSV: {e}")
        print("=" * 70)
        return

    # --- 2. Amostra dos Dados (Head) ---
    print_header("1. Amostra dos Dados (Primeiras 5 Linhas)")
    # .to_string() formata o DataFrame como uma tabela de texto legível
    print(df.head().to_string())

    # --- 3. Informações do DataFrame (Info) ---
    print_header("2. Informações do DataFrame (Tipos de Coluna e Nulos)")
    # O df.info() imprime diretamente. Para capturá-lo e formatá-lo,
    # usamos um buffer de string.
    buffer = io.StringIO()
    df.info(buf=buffer)
    info_str = buffer.getvalue()
    print(info_str)

    # --- 4. Contagem de Valores Nulos ---
    print_header("3. Resumo de Valores Nulos por Coluna")
    null_counts = df.isnull().sum()
    
    if null_counts.sum() == 0:
        print("Ótimo! Não há valores nulos em nenhuma coluna.")
    else:
        # Filtra para mostrar apenas colunas que *possuem* valores nulos
        print(null_counts[null_counts > 0].to_string())

    # --- 5. Estatísticas Descritivas ---
    print_header("4. Estatísticas Descritivas (Colunas Numéricas)")
    # O 'include='np.number' garante que só analisará colunas numéricas
    # .to_string() formata a saída para melhor visualização no log
    try:
        print(df.describe().to_string())
    except Exception as e:
        print(f"Não foi possível calcular estatísticas descritivas: {e}")

    # --- 6. Análise de Tickers ---
    print_header("5. Análise da Coluna 'ticker'")
    if 'ticker' in df.columns:
        unique_tickers = df['ticker'].unique()
        num_unique_tickers = len(unique_tickers)
        print(f"Total de tickers únicos encontrados: {num_unique_tickers}")
        
        # Mostra uma amostra se houver muitos tickers
        if num_unique_tickers > 10:
            print(f"Amostra de tickers: {unique_tickers[:10]}...")
        else:
            print(f"Tickers presentes: {unique_tickers}")
    else:
        print("Coluna 'ticker' não encontrada.")

    print("\n" + "=" * 70)
    print("--- ANÁLISE CONCLUÍDA ---")
    print("=" * 70)

# --- Ponto de Execução Principal ---
if __name__ == "__main__":
    
    # Define o caminho do arquivo
    path_do_arquivo = 'data/historical/all_histories.csv'
    
    # Executa a função de análise
    analyze_dataframe(path_do_arquivo)

--- Iniciando Análise do Arquivo: data/historical/all_histories.csv ---

[SUCESSO] Arquivo carregado. Total de 357154 linhas e 10 colunas.

 1. AMOSTRA DOS DADOS (PRIMEIRAS 5 LINHAS) 
     ticker       date      Open      High       Low     Close  Adj Close     Volume  Dividends  Stock Splits
0  ABEV3.SA 2011-01-03  8.632311  8.728203  8.630313  8.690246   4.694568   576145.0        0.0           0.0
1  ABEV3.SA 2011-01-04  8.784141  8.784141  8.630313  8.692244   4.695646   328368.0        0.0           0.0
2  ABEV3.SA 2011-01-05  8.672266  8.718215  8.448517  8.530425   4.608232   299836.0        0.0           0.0
3  ABEV3.SA 2011-01-06  8.560392  8.590358  8.396576  8.450515   4.565063   731319.0        0.0           0.0
4  ABEV3.SA 2011-01-07  8.450515  8.550403  8.368607  8.416553   4.546715  1090222.0        0.0           0.0

 2. INFORMAÇÕES DO DATAFRAME (TIPOS DE COLUNA E NULOS) 
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 357154 entries, 0 to 357153
Data columns (total 1

In [5]:
import pandas as pd
import numpy as np
import logging
import os # Garante que 'os' está importado

# Configuração de logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# --- 1. Carregar os Dados ---
try:
    # Tenta carregar o Parquet primeiro, é mais rápido
    df_prices_raw = pd.read_parquet("data/historical/all_histories.parquet")
    logger.info(f"Dados de preço brutos carregados do PARQUET: {df_prices_raw.shape[0]} linhas")
except FileNotFoundError:
    logger.warning("Arquivo all_histories.parquet não encontrado! Tentando carregar o CSV...")
    try:
        # Carrega o CSV como fallback
        df_prices_raw = pd.read_csv("data/historical/all_histories.csv", parse_dates=['date'])
        logger.info(f"Dados de preço brutos carregados do CSV: {df_prices_raw.shape[0]} linhas")
    except FileNotFoundError as e:
        logger.error("Nenhum arquivo de histórico (parquet ou csv) encontrado. Execute o download primeiro.")
        raise e
except Exception as e:
    logger.error(f"Erro ao carregar dados: {e}")
    raise

# --- 2. Limpeza Crítica (Remover NaN de pré-IPO) ---
# Remove todas as linhas onde a ação ainda não existia (Adj Close ou Volume são NaN)
df_prices_clean = df_prices_raw.dropna(subset=['Adj Close', 'Volume'])

# --- 3. Limpeza Opcional (Remover dias sem negociação) ---
df_prices_clean = df_prices_clean[df_prices_clean['Volume'] > 0]

linhas_removidas = len(df_prices_raw) - len(df_prices_clean)
logger.info(f"Dados de preço limpos: {df_prices_clean.shape[0]} linhas (removidas {linhas_removidas} linhas com NaN ou Volume 0)")

# --- 4. Salvar os Arquivos Limpos ---
# Garante que o diretório existe
output_dir = "data/historical"
os.makedirs(output_dir, exist_ok=True)

parquet_path = os.path.join(output_dir, "all_histories_cleaned.parquet")
csv_path = os.path.join(output_dir, "all_histories_cleaned.csv")

try:
    # Salvar em Parquet (preferencial para o próximo passo)
    df_prices_clean.to_parquet(parquet_path, index=False)
    logger.info(f"✅ Arquivo limpo salvo em (Parquet): {parquet_path}")

    # Salvar em CSV (para sua verificação)
    # date_format garante que a data seja salva em formato legível
    df_prices_clean.to_csv(csv_path, index=False, date_format='%Y-%m-%d')
    logger.info(f"✅ Arquivo limpo salvo em (CSV): {csv_path}")

except Exception as e:
    logger.error(f"❌ Erro ao salvar arquivos limpos: {e}")

print(f"\nLimpeza concluída. {linhas_removidas} linhas de 'lookahead' removidas.")
print(df_prices_clean.head())


Limpeza concluída. 79964 linhas de 'lookahead' removidas.
     ticker       date      Open      High       Low     Close  Adj Close  \
0  ABEV3.SA 2011-01-03  8.632311  8.728203  8.630313  8.690246   4.694568   
1  ABEV3.SA 2011-01-04  8.784141  8.784141  8.630313  8.692244   4.695646   
2  ABEV3.SA 2011-01-05  8.672266  8.718215  8.448517  8.530425   4.608232   
3  ABEV3.SA 2011-01-06  8.560392  8.590358  8.396576  8.450515   4.565063   
4  ABEV3.SA 2011-01-07  8.450515  8.550403  8.368607  8.416553   4.546715   

      Volume  Dividends  Stock Splits  
0   576145.0        0.0           0.0  
1   328368.0        0.0           0.0  
2   299836.0        0.0           0.0  
3   731319.0        0.0           0.0  
4  1090222.0        0.0           0.0  


In [14]:
import pandas as pd
import numpy as np
import logging
import os

# --- Configuração ---
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

INPUT_FILE = "data/historical/all_histories_cleaned.parquet"
OUTPUT_DIR = "data/historical"

def gerar_precos_pivotados():
    logger.info("Iniciando o Passo 1: Geração dos Preços Pivotados (Wide)...")
    
    # 1. Carregar os dados de preço limpos
    try:
        df_prices_clean = pd.read_parquet(INPUT_FILE)
        logger.info(f"Dados limpos '{INPUT_FILE}' carregados.")
    except FileNotFoundError:
        logger.error(f"ARQUIVO NÃO ENCONTRADO: {INPUT_FILE}")
        logger.error("Execute o 'Script 1: Limpeza' primeiro.")
        return

    df_prices_clean['date'] = pd.to_datetime(df_prices_clean['date'])

    # 2. Criar a base de preços mensais
    logger.info("Reamostrando para Frequência Mensal (M e MS)...")
    # 2a. Fechamento (Último dia do mês 'M')
    df_prices_mensal_raw = df_prices_clean.set_index('date').groupby('ticker').resample('M').last()

    # 2b. Abertura (Primeiro dia do mês 'MS')
    df_open_mensal_raw = df_prices_clean.set_index('date').groupby('ticker').resample('MS').first()

    # 3. Pivotar para o formato "wide"
    logger.info("Corrigindo MultiIndex e Pivotando...")
    
    # CORREÇÃO: Dropar a coluna 'ticker' duplicada antes de resetar o índice
    df_prices_mensal_long = df_prices_mensal_raw.drop(columns='ticker', errors='ignore').reset_index()
    df_open_mensal_long = df_open_mensal_raw.drop(columns='ticker', errors='ignore').reset_index()

    # Agora o .pivot() funcionará
    df_close_wide = df_prices_mensal_long.pivot(index='date', columns='ticker', values='Adj Close')
    df_open_wide = df_open_mensal_long.pivot(index='date', columns='ticker', values='Open')

    # 4. Sincronizar os índices de data
    logger.info("Sincronizando e preenchendo índices de data...")
    idx_union = df_close_wide.index.union(df_open_wide.index)
    
    df_close_wide = df_close_wide.reindex(idx_union, method='ffill')
    df_open_wide = df_open_wide.reindex(idx_union, method='ffill')

    # Preenche NaNs (pré-IPO e pós-delist)
    df_close_wide = df_close_wide.ffill().bfill()
    df_open_wide = df_open_wide.ffill().bfill()

    # --- 5. SALVAR OS ARQUIVOS FALTOSOS ---
    os.makedirs(OUTPUT_DIR, exist_ok=True)
    
    path_close = os.path.join(OUTPUT_DIR, "prices_close_wide.parquet")
    path_open = os.path.join(OUTPUT_DIR, "prices_open_wide.parquet")
    
    df_close_wide.to_parquet(path_close)
    df_open_wide.to_parquet(path_open)
    
    logger.info(f"✅ ARQUIVO FALTOSO GERADO: {path_close}")
    logger.info(f"✅ ARQUIVO FALTOSO GERADO: {path_open}")
    print("\n--- Amostra de Fechamento (Wide) ---")
    print(df_close_wide.tail())

if __name__ == "__main__":
    gerar_precos_pivotados()


--- Amostra de Fechamento (Wide) ---
ticker      ABEV3.SA   ALOS3.SA  ANIM3.SA  ASAI3.SA  AURE3.SA   AZZA3.SA  \
date                                                                       
2025-08-31     12.35  23.764782      3.37     10.52     10.86  34.709999   
2025-09-01     12.35  23.764782      3.37     10.52     10.86  34.709999   
2025-09-30     12.09  25.721401      3.43      9.51     10.27  30.150000   
2025-10-01     12.09  25.721401      3.43      9.51     10.27  30.150000   
2025-10-31     12.12  24.480000      3.22      8.15     11.70  27.540001   

ticker       BBAS3.SA   BBDC3.SA   BBDC4.SA   BBSE3.SA  ...   TOTS3.SA  \
date                                                    ...              
2025-08-31  21.389999  14.151272  16.512844  32.820000  ...  42.957096   
2025-09-01  21.389999  14.151272  16.512844  32.820000  ...  42.957096   
2025-09-30  22.090000  15.212403  17.670698  33.259998  ...  45.930000   
2025-10-01  22.090000  15.212403  17.670698  33.259998  ...

ERROR:__main__:ERRO CRÍTICO: As colunas de fundamento chave ['ROIC'] não foram encontradas após o merge!
ERROR:__main__:Verifique os nomes das colunas no arquivo de fundamentos e se o merge_asof funcionou (veja logs acima).


In [9]:
import pandas as pd
import numpy as np
import os
import logging

# Configuração de logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

logger.info("Iniciando Passo 2: Cálculo da Volatilidade...")

# Carregar os dados de preço DIÁRIOS (limpos)
df_prices_daily = pd.read_parquet("data/historical/all_histories_cleaned.parquet")
df_prices_daily['date'] = pd.to_datetime(df_prices_daily['date'])

# 1. Calcular retornos diários
df_prices_daily['returns'] = df_prices_daily.groupby('ticker')['Adj Close'].pct_change()

# 2. Calcular a volatilidade móvel de 63 dias (~3 meses de negociação)
# .std() calcula o desvio padrão (volatilidade)
# reset_index(0, drop=True) é necessário após o .rolling() em um groupby
df_prices_daily['VOLATILIDADE'] = df_prices_daily.groupby('ticker')['returns'].rolling(window=63).std().reset_index(0, drop=True)

# 3. Resample da volatilidade para MENSAL (pegamos o último valor do mês)
df_vol_mensal = df_prices_daily.set_index('date').groupby('ticker').resample('M').last()['VOLATILIDADE'].reset_index()

logger.info(f"✅ Volatilidade mensal calculada. {len(df_vol_mensal)} registros.")
print(df_vol_mensal.tail())

         ticker       date  VOLATILIDADE
13681  YDUQ3.SA 2025-06-30      0.032834
13682  YDUQ3.SA 2025-07-31      0.029565
13683  YDUQ3.SA 2025-08-31      0.026134
13684  YDUQ3.SA 2025-09-30      0.027915
13685  YDUQ3.SA 2025-10-31      0.025774


In [10]:
logger.info("Iniciando Passo 3: Agregação de Sentimento...")

# Carregar seu arquivo de notícias com sentimento
# (Vou usar o nome do seu screenshot)
df_sent_raw = pd.read_parquet("data/news/news_with_sentiment.parquet")

# 1. Renomear colunas para padronização
df_sent_raw = df_sent_raw.rename(columns={
    'ticker_query': 'ticker',
    'published_date': 'date'
})
df_sent_raw['date'] = pd.to_datetime(df_sent_raw['date'], utc=True)
df_sent_raw['date'] = df_sent_raw['date'].dt.tz_localize(None) # Remover timezone para o merge

# 2. Garantir que o ticker tem o sufixo .SA (se necessário)
# Se seu ticker_query for "PETR4", esta linha ajusta para "PETR4.SA"
if not df_sent_raw['ticker'].str.contains('.SA').any():
    df_sent_raw['ticker'] = df_sent_raw['ticker'].apply(lambda x: f"{x}.SA" if not x.endswith(".SA") else x)

# 3. Agregar por ticker e Mês
df_sent_mensal = df_sent_raw.set_index('date').groupby('ticker').resample('M').agg(
    SENTIMENT_MEDIO=('numeric_sentiment', 'mean'), # Coluna que você mencionou
    SENTIMENT_STD=('numeric_sentiment', 'std'),
    NEWS_COUNT=('ticker', 'count')
).reset_index()

logger.info(f"✅ Sentimento mensal agregado. {len(df_sent_mensal)} registros.")
print(df_sent_mensal.tail())

    ticker       date  SENTIMENT_MEDIO  SENTIMENT_STD  NEWS_COUNT
183  VIVT3 2025-09-30         0.333333       0.577350           3
184  VIVT3 2025-10-31         0.000000       0.000000          19
185  WEGE3 2025-09-30         0.085714       0.373491          35
186  WEGE3 2025-10-31         0.127660       0.396562          47
187  YDUQ3 2025-09-30         0.142857       0.377964           7


In [23]:
import pandas as pd
import numpy as np
import logging
import os

# (Execute os Passos 2 e 3 acima para ter 'df_vol_mensal' e 'df_sent_mensal')

logger.info("Iniciando Passo 4: Unificação do DataFrame Mestre...")

# --- 1. Carregar a Base de Preços Mensal ---
# Vamos usar o df_close_wide que você já criou e "des-pivotar" (melt)
try:
    df_close_wide = pd.read_parquet("data/historical/prices_close_wide.parquet")
    df_base_mensal = df_close_wide.melt(ignore_index=False, var_name='ticker', value_name='Adj Close').reset_index()
    df_base_mensal = df_base_mensal.rename(columns={'index': 'date'})
    logger.info(f"Base de preços mensal carregada: {len(df_base_mensal)} registros (tickers * meses)")
except FileNotFoundError:
    logger.error("Execute o script de Pivot (Script 2) primeiro!")
    raise

# --- 2. Carregar Fundamentos (Trimestrais) ---
df_fund = pd.read_parquet("data/aurum_scores_output/aurum_quality_scores_complete.parquet")
df_fund = df_fund.rename(columns={'DT_FIM_EXERC': 'date'})
df_fund['date'] = pd.to_datetime(df_fund['date'])

# --- 3. Carregar o Mapeamento (DE-PARA) ---
try:
    df_mapping = pd.read_csv("ticker_cnpj_map.csv") # O arquivo que você criou no Passo 1
    # Limpar CNPJs (remover pontuação) se necessário
    # df_mapping['CNPJ_CIA'] = df_mapping['CNPJ_CIA'].str.replace(r'[^\d]', '', regex=True)
except FileNotFoundError:
    logger.error("Arquivo 'ticker_cnpj_map.csv' não encontrado! Crie-o antes de continuar.")
    raise

# Juntar o ticker aos dados de fundamento
df_fund_com_ticker = pd.merge(df_fund, df_mapping, on='CNPJ_CIA', how='left')
df_fund_com_ticker = df_fund_com_ticker.dropna(subset=['ticker'])
logger.info("Fundamentos mapeados para tickers.")

# --- 4. Construir o DataFrame Mestre ---
# Ordenar tudo por data é crucial para o merge_asof
df_master = df_base_mensal.sort_values(by='date')
df_fund_com_ticker = df_fund_com_ticker.sort_values(by='date')

# 4a. Juntar Fundamentos (Trimestrais)
# "Para cada mês na minha base, pegue o último balanço disponível"
df_master = pd.merge_asof(
    df_master,
    df_fund_com_ticker,
    on='date',
    by='ticker',
    direction='backward' # 'backward' = olhar para trás
)
logger.info("Merge 'as-of' dos fundamentos concluído.")

# 4b. Juntar Volatilidade (Mensal)
df_master = pd.merge(
    df_master,
    df_vol_mensal,
    on=['date', 'ticker'],
    how='left'
)
logger.info("Merge da volatilidade concluído.")

# 4c. Juntar Sentimento (Mensal)
df_master = pd.merge(
    df_master,
    df_sent_mensal,
    on=['date', 'ticker'],
    how='left'
)
logger.info("Merge do sentimento concluído.")

# --- 5. Limpeza Final ---
# Preencher NaNs de sentimento (meses sem notícias) com Neutro (0)
df_master['SENTIMENT_MEDIO'] = df_master['SENTIMENT_MEDIO'].fillna(0)

# **MUITO IMPORTANTE**: Remover linhas onde os fundamentos ainda não existiam
# (ex: antes do IPO da ação ou antes dos dados de 2011)
# Se 'ROE' é nulo, significa que o merge_asof não encontrou fundamentos para aquele mês.
df_master = df_master.dropna(subset=['ROE', 'ROIC']) # Use as colunas chave do seu score

logger.info(f"Limpeza final concluída. DataFrame Mestre pronto com {len(df_master)} linhas.")

# --- 6. Salvar o Novo DataFrame Mestre ---
output_path = "data/aurum_master_features.parquet"
df_master.to_parquet(output_path, index=False)
logger.info(f"✅✅✅ DataFrame Mestre salvo em: {output_path} ✅✅✅")
print(df_master.info())

FileNotFoundError: [Errno 2] No such file or directory: 'data/aurum_scores_output/aurum_quality_scores_complete.parquet'

In [7]:
import pandas as pd
import numpy as np
import logging
import os # Garante que 'os' está importado

# Configuração de logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# 1. Carregar os dados de preço limpos
try:
    df_prices_clean = pd.read_parquet("data/historical/all_histories_cleaned.parquet")
    logger.info("Dados limpos 'all_histories_cleaned.parquet' carregados.")
except FileNotFoundError:
    logger.error("Arquivo limpo não encontrado. Execute o Script 1 (Limpeza) primeiro.")
    # Parar a execução se o arquivo não existir
    raise

df_prices_clean['date'] = pd.to_datetime(df_prices_clean['date'])

# 2. Criar a base de preços mensais
# 2a. Fechamento (Usamos .last() no fim do mês 'M')
df_prices_mensal_raw = df_prices_clean.set_index('date').groupby('ticker').resample('M').last()

# 2b. Abertura (Usamos .first() no início do mês 'MS')
df_open_mensal_raw = df_prices_clean.set_index('date').groupby('ticker').resample('MS').first()


# 3. Pivotar para o formato "wide"

# <<< A CORREÇÃO ESTÁ AQUI >>>
# O .last() e .first() mantiveram a coluna 'ticker' original,
# mas 'ticker' também está no MultiIndex. Isso causa um conflito.
# Vamos dropar a coluna 'ticker' redundante ANTES de resetar o índice.
# 'errors="ignore"' evita erro caso a coluna não exista.

df_prices_mensal_long = df_prices_mensal_raw.drop(columns='ticker', errors='ignore').reset_index()
df_open_mensal_long = df_open_mensal_raw.drop(columns='ticker', errors='ignore').reset_index()


# Agora o .pivot() funcionará
logger.info("Pivotando dados de Fechamento...")
df_close_wide = df_prices_mensal_long.pivot(index='date', columns='ticker', values='Adj Close')

logger.info("Pivotando dados de Abertura...")
df_open_wide = df_open_mensal_long.pivot(index='date', columns='ticker', values='Open')


# 4. Sincronizar os índices de data (MUITO IMPORTANTE)
# Garante que ambos os DataFrames (open e close) cubram o mesmo período
logger.info("Sincronizando índices de data...")
idx_union = df_close_wide.index.union(df_open_wide.index)

# .reindex() com 'ffill' (forward-fill) preenche o preço de fechamento
# nos dias de início de mês (MS) que não existiam no df_close_wide (M)
df_close_wide = df_close_wide.reindex(idx_union, method='ffill')
df_open_wide = df_open_wide.reindex(idx_union, method='ffill')

# Preenche os NaNs iniciais (antes do IPO) que podem ter sido reintroduzidos
# E também os finais, caso uma ação seja deslistada (usa bfill)
df_close_wide = df_close_wide.ffill().bfill()
df_open_wide = df_open_wide.ffill().bfill()

logger.info("✅ DataFrames de Preço (Wide) prontos para o backtest.")

print("\nDataFrame de Fechamento (Wide) - Amostra:")
print(df_close_wide.tail())

print("\nDataFrame de Abertura (Wide) - Amostra:")
print(df_open_wide.tail())

# --- 5. SALVAR OS ARQUIVOS WIDE ---
# Salvar os dataframes pivotados para uso fácil no próximo script
output_dir = "data/historical"
os.makedirs(output_dir, exist_ok=True) # Garante que o diretório existe

df_close_wide.to_parquet(os.path.join(output_dir, "prices_close_wide.parquet"))
df_open_wide.to_parquet(os.path.join(output_dir, "prices_open_wide.parquet"))

logger.info(f"💾 DataFrames de preço pivotados salvos em {output_dir}")

C:\Users\kaike\AppData\Local\Temp\ipykernel_1924\2068620262.py:23: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  df_prices_mensal_raw = df_prices_clean.set_index('date').groupby('ticker').resample('M').last()
C:\Users\kaike\AppData\Local\Temp\ipykernel_1924\2068620262.py:23: FutureWarning: DataFrameGroupBy.resample operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_prices_mensal_raw = df_prices_clean.set_index('date').groupby('ticker').resample('M').last()



DataFrame de Fechamento (Wide) - Amostra:
ticker      ABEV3.SA   ALOS3.SA  ANIM3.SA  ASAI3.SA  AURE3.SA   AZZA3.SA  \
date                                                                       
2025-08-31     12.35  23.764782      3.37     10.52     10.86  34.709999   
2025-09-01     12.35  23.764782      3.37     10.52     10.86  34.709999   
2025-09-30     12.09  25.721401      3.43      9.51     10.27  30.150000   
2025-10-01     12.09  25.721401      3.43      9.51     10.27  30.150000   
2025-10-31     12.12  24.480000      3.22      8.15     11.70  27.540001   

ticker       BBAS3.SA   BBDC3.SA   BBDC4.SA   BBSE3.SA  ...   TOTS3.SA  \
date                                                    ...              
2025-08-31  21.389999  14.151272  16.512844  32.820000  ...  42.957096   
2025-09-01  21.389999  14.151272  16.512844  32.820000  ...  42.957096   
2025-09-30  22.090000  15.212403  17.670698  33.259998  ...  45.930000   
2025-10-01  22.090000  15.212403  17.670698  33.259998

C:\Users\kaike\AppData\Local\Temp\ipykernel_1924\2068620262.py:26: FutureWarning: DataFrameGroupBy.resample operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_open_mensal_raw = df_prices_clean.set_index('date').groupby('ticker').resample('MS').first()


### 📊 Documentação dos DataFrames da CVM (DFP / ITR)

Este documento descreve os **DataFrames gerados a partir dos arquivos públicos da CVM** — especificamente, os conjuntos **BPP**, **BPA** e **DRE**, extraídos dos demonstrativos financeiros enviados pelas companhias abertas brasileiras.

Cada DataFrame representa **uma parte diferente das demonstrações contábeis** disponibilizadas pela CVM (Comissão de Valores Mobiliários), e é resultado do processamento dos arquivos `.csv` baixados e descompactados pelo script `cvm_downloader_clean.py`.

---

#### 🧱 Estrutura e Função de Cada DataFrame

### **1. `raw_bpp` — Balanço Patrimonial Passivo**
O DataFrame `raw_bpp` contém as contas e valores que compõem o **lado do passivo** no balanço patrimonial das companhias abertas.  
Ele representa as **obrigações, patrimônio líquido e contas de financiamento** de curto e longo prazo.

**Função:** permite analisar a **estrutura de capital e endividamento** das empresas.

---

#### **2. `raw_bpa` — Balanço Patrimonial Ativo**
O DataFrame `raw_bpa` contém as contas e valores que compõem o **lado do ativo** do balanço patrimonial.  
Inclui **ativos circulantes e não circulantes**, como caixa, contas a receber, estoques e imobilizado.

**Função:** permite estudar a **composição dos ativos e a liquidez** das companhias.

---

#### **3. `raw_dre` — Demonstração do Resultado do Exercício**
O DataFrame `raw_dre` representa a **Demonstração do Resultado do Exercício**, mostrando as receitas, custos, despesas e o lucro ou prejuízo de um período.

**Função:** possibilita a **análise de desempenho e rentabilidade** da empresa.

---

### 🧾 Dicionário de Colunas (comum aos três DataFrames)

| Coluna | Tipo | Descrição |
|:-------|:-----|:-----------|
| **CNPJ_CIA** | string | CNPJ da companhia aberta que enviou o demonstrativo. Identificador único da empresa. |
| **DT_REFER** | date | Data de referência do relatório entregue à CVM (geralmente o último dia do período reportado). |
| **VERSAO** | int | Versão do documento entregue à CVM (1 = primeira entrega, >1 = reenvio/correção). |
| **DENOM_CIA** | string | Nome completo da companhia conforme cadastro na CVM. |
| **CD_CVM** | string/int | Código numérico da companhia na base da CVM (identificador único). |
| **GRUPO_DFP** | string | Indica o grupo de demonstrativo (ex.: DFP = Demonstrações Financeiras Padronizadas). |
| **MOEDA** | string | Moeda de apresentação dos valores (ex.: 'REAL', 'USD'). |
| **ESCALA_MOEDA** | string | Escala dos valores informados (ex.: 'UNIDADE', 'MIL', 'MILHÃO'). |
| **ORDEM_EXERC** | string | Indica se o exercício é 'ÚLTIMO' (ano mais recente) ou 'PENÚLTIMO' (ano anterior). |
| **DT_FIM_EXERC** | date | Data de encerramento do exercício financeiro (geralmente 31/12/AAAA). |
| **CD_CONTA** | string | Código padronizado da conta contábil conforme o plano da CVM. |
| **DS_CONTA** | string | Descrição textual da conta contábil (ex.: “Ativo Circulante”, “Lucro Líquido do Exercício”). |
| **VL_CONTA** | float | Valor numérico registrado na conta contábil (em função da escala da moeda). |
| **ST_CONTA_FIXA** | string | Indicador se a conta é fixa (‘S’) ou variável (‘N’) no plano contábil. |

---

### 🧩 Colunas específicas por DataFrame

#### 🔹 `raw_bpp` (Passivo)
Mesmas colunas da estrutura comum.  
O foco está nas contas de **passivos** e **patrimônio líquido**, como:
- Fornecedores  
- Empréstimos e financiamentos  
- Obrigações fiscais  
- Capital social  
- Lucros acumulados  

---

#### 🔹 `raw_bpa` (Ativo)
Mesmas colunas da estrutura comum.  
O foco está nas contas de **ativos**, como:
- Caixa e equivalentes  
- Contas a receber  
- Estoques  
- Imobilizado  
- Investimentos  

---

#### 🔹 `raw_dre` (Resultado)
Inclui **duas datas adicionais**:

| Coluna | Tipo | Descrição |
|:-------|:-----|:-----------|
| **DT_INI_EXERC** | date | Data de início do exercício (geralmente 01/01/AAAA). |
| **DT_FIM_EXERC** | date | Data de término do exercício (geralmente 31/12/AAAA). |

Essas datas definem o **período de apuração** do resultado.

Principais contas encontradas:
- Receita líquida de vendas  
- Custo dos produtos vendidos  
- Despesas operacionais  
- Resultado financeiro  
- Lucro (ou prejuízo) do exercício  

---

### 📦 Relação entre os DataFrames

| DataFrame | Tipo de Demonstração | Relação com outros |
|:-----------|:--------------------|:-------------------|
| **raw_bpa** | Ativo (Balanço Patrimonial) | Relaciona-se com `raw_bpp` (mesmo período e CNPJ_CIA) |
| **raw_bpp** | Passivo (Balanço Patrimonial) | Complementa o `raw_bpa` para totalizar o balanço |
| **raw_dre** | Resultado (DRE) | Demonstra o desempenho que afeta o patrimônio líquido do `raw_bpp` |


### 📈 Uso típico no pipeline

1. **Download e extração** via `cvm_downloader_clean.py` → gera os CSVs.
2. **Leitura** dos arquivos CSV em pandas → cria `raw_bpp`, `raw_bpa`, `raw_dre`.
3. **Limpeza e padronização** → converte tipos, remove duplicatas, ajusta nomes.
4. **Enriquecimento e análise** → cálculo de indicadores como liquidez, endividamento, rentabilidade.


**Referência oficial:**  
📚 [Portal de Dados Abertos CVM – Documentos DFP/ITR](https://dados.cvm.gov.br/dados/CIA_ABERTA/DOC/)




---

### **📦 cvm_downloader** — Download & Unzip automático (CVM: DFP / ITR)

**1. Descrição curta**  
Script robusto para baixar (streaming com progress bar) e descompactar os arquivos públicos da CVM (`DFP` e `ITR`) por ano. Inclui retries, backoff, gravação segura (.part → rename) e verificação para pular arquivos já existentes. Tem suporte a execução sequencial ou com --workers (thread pool).

**2. Principais responsabilidades**
- Construir URLs padrão: **https://dados.cvm.gov.br/dados/CIA_ABERTA/DOC/{doc_type}/DADOS/{doc_type}_cia_aberta_{year}.zip**
- Baixar zips com **requests.Session** + retries
- Exibir barra de progresso (**tqdm**)
- Descompactar zips (**zipfile.ZipFile**)
- Pular arquivos já baixados / extraídos
- Opcional: paralelizar downloads via **ThreadPoolExecutor**

**3. Entradas / Parâmetros**
- --doc-types (ex.: DFP,ITR)
- --start-year / --end-year (ex.: 2011 / 2025)
- --workers (nº threads; default 1)

**4. Saídas**
- data/cvm/zip/{DOC}_cia_aberta_{YEAR}.zip
- data/cvm/unzipped/{DOC}_{YEAR}/... (conteúdo extraído)
- Retorna resumo (ok / skipped / failed) e código de saída CLI

---


In [ ]:
# --- Configuração ---
# Onde os dados serão salvos
BASE_DIR = os.path.join("data", "cvm")
ZIP_DIR = os.path.join(BASE_DIR, "zip")
UNZIPPED_DIR = os.path.join(BASE_DIR, "unzipped")

# Cria as pastas se não existirem
os.makedirs(ZIP_DIR, exist_ok=True)
os.makedirs(UNZIPPED_DIR, exist_ok=True)

# URL base para os arquivos da CVM
URL_BASE = "https://dados.cvm.gov.br/dados/CIA_ABERTA/DOC/{doc_type}/DADOS/"

# Tipos de documentos que queremos
# DFP -> Anual, ITR -> Trimestral
DOC_TYPES = ['DFP', 'ITR']

YEARS = range(2011, 2026) 

def download_and_unzip(url, zip_path, unzipped_path):
    """Baixa e descompacta um arquivo ZIP se ele não existir localmente."""
    if os.path.exists(zip_path):
        print(f"Arquivo já existe, pulando download: {os.path.basename(zip_path)}")
    else:
        print(f"Baixando: {os.path.basename(zip_path)}")
        try:
            response = requests.get(url, stream=True)
            response.raise_for_status() # Lança erro se a requisição falhar
            
            total_size = int(response.headers.get('content-length', 0))
            
            with open(zip_path, 'wb') as f, tqdm(
                desc=os.path.basename(zip_path),
                total=total_size,
                unit='iB',
                unit_scale=True,
                unit_divisor=1024,
            ) as bar:
                for data in response.iter_content(chunk_size=1024):
                    size = f.write(data)
                    bar.update(size)
            print("Download completo.")
        except requests.exceptions.RequestException as e:
            print(f"Erro no download de {url}: {e}")
            return # Sai da função se o download falhar

    print(f"Descompactando: {os.path.basename(zip_path)}")
    try:
        with ZipFile(zip_path, 'r') as zip_ref:
            zip_ref.extractall(unzipped_path)
        print("Descompactado com sucesso.")
    except Exception as e:
        print(f"Erro ao descompactar {zip_path}: {e}")

if __name__ == "__main__":
    for doc_type in DOC_TYPES:
        for year in YEARS:
            filename = f"{doc_type}_cia_aberta_{year}.zip"
            url = URL_BASE.format(doc_type=doc_type) + filename
            
            zip_path = os.path.join(ZIP_DIR, filename)
            unzipped_path = os.path.join(UNZIPPED_DIR, f"{doc_type}_{year}")
            
            download_and_unzip(url, zip_path, unzipped_path)
            print("-" * 50)
            
    print("\nProcesso de download e extração concluído!")


---
### 🧩 cvm_parser — Parser / Busca consolidada (_con_ files) e preparação

**1. Descrição curta**  
Pipeline para localizar arquivos CSV consolidados _con_*.csv extraídos da CVM, aplicar filtros (ÚLTIMO/PENÚLTIMO, escala MIL/UNIDADE), normalizar VL_CONTA, diagnosticar CNPJ(s) de interesse (ex.: Sanepar), concatenar, deduplicar e salvar resultado processado.

**2. Principais responsabilidades**
- Encontrar arquivos consolidados recursivamente (`UNZIPPED_DIR/**/*_con_*.csv`)
- Ler CSVs com pd.read_csv(..., encoding='latin-1', sep=';')
- Filtrar por ORDEM_EXERC (`ÚLTIMO` / `PENÚLTIMO`) e ESCALA_MOEDA (`MIL` / `UNIDADE`)
- Converter VL_CONTA para numérico e aplicar ajuste (multiplica por 1000 quando `ESCALA_MOEDA == 'MIL'`)
- Diagnóstico: detectar CNPJ_SAPR e imprimir valores de `ORDEM_EXERC` / `ESCALA_MOEDA`
- Concatenar chunks, ordenar por CNPJ_CIA, DT_FIM_EXERC, VERSAO
- Drop duplicates por ['CNPJ_CIA','DT_FIM_EXERC','CD_CONTA']
- Salvar processed/raw_{doc}.parquet e processed/raw_{doc}.csv

**3. Entradas**
- data/cvm/unzipped/... (arquivos _con_*.csv)

**4. Saídas**
- data/cvm/processed/raw_{dre|bpa|bpp}.parquet
- data/cvm/processed/raw_{dre|bpa|bpp}.csv

---

In [ ]:
# -----------------------
# Configuração (edite aqui)
# -----------------------
BASE_DIR = Path("data") / "cvm"
UNZIPPED_DIR = BASE_DIR / "unzipped"
PROCESSED_DIR = BASE_DIR / "processed"
PROCESSED_DIR.mkdir(parents=True, exist_ok=True)

DOC_PATTERNS_BROAD = {
    "dre": "*_DRE_con_*.csv",
    "bpa": "*_BPA_con_*.csv",
    "bpp": "*_BPP_con_*.csv",
}

CNPJ_SAPR = "76.484.013/0001-45"
CNPJs_DEBUG = [CNPJ_SAPR, "33.839.910/0001-11"]  # VIVA3

CATEGORY_COLS = [
    "CNPJ_CIA", "DENOM_CIA", "GRUPO_DFP", "MOEDA",
    "ESCALA_MOEDA", "ORDEM_EXERC", "CD_CONTA",
    "DS_CONTA", "ST_CONTA_FIXA"
]

# -----------------------
# Logging
# -----------------------
logging.basicConfig(level=logging.INFO, format="%(asctime)s %(levelname)s %(message)s")
logger = logging.getLogger("cvm_parser")

# -----------------------
# Helpers
# -----------------------
def find_consolidated_files(unzipped_dir: Path, pattern: str) -> List[Path]:
    """Retorna lista de arquivos que batem com o padrão (recursivo)."""
    return list(unzipped_dir.rglob(pattern))


def _ensure_category(df: pd.DataFrame, col: str) -> None:
    """Converte coluna para category quando apropriado (silencioso)."""
    try:
        if col in df.columns and not pd.api.types.is_categorical_dtype(df[col]):
            df[col] = df[col].astype("category")
    except Exception:
        # não quebrar a execução por problemas de conversão
        pass


def _read_csv_safe(path: Path) -> Optional[pd.DataFrame]:
    """Lê um CSV com parâmetros padrão usados no pipeline; captura exceções e retorna None se falhar."""
    try:
        df = pd.read_csv(path, encoding="latin-1", sep=";", low_memory=False, dtype={"CNPJ_CIA": str})
        return df
    except Exception as exc:
        logger.warning("Falha lendo %s: %s", path, exc)
        return None


def _process_df_chunk(df: pd.DataFrame) -> Optional[pd.DataFrame]:
    """
    Aplicar filtros e tratamentos ao chunk (já lido).
    Retorna DataFrame filtrado pronto para concatenar ou None se vazio.
    """
    # garantir colunas chave
    if "DT_FIM_EXERC" not in df.columns:
        return None

    # normalizar e otimizar
    for col in CATEGORY_COLS:
        if col in df.columns:
            _ensure_category(df, col)

    # converter data e remover linhas sem data válida
    df["DT_FIM_EXERC"] = pd.to_datetime(df["DT_FIM_EXERC"], errors="coerce")
    df = df.dropna(subset=["DT_FIM_EXERC"])
    if df.empty:
        return None

    # aplicar filtro ORDEM_EXERC & ESCALA_MOEDA quando existirem
    if "ORDEM_EXERC" in df.columns and "ESCALA_MOEDA" in df.columns:
        mask = (
            df["ORDEM_EXERC"].isin(["ÚLTIMO", "PENÚLTIMO"]) &
            df["ESCALA_MOEDA"].isin(["MIL", "UNIDADE"])
        )
        df = df.loc[mask].copy()

    if df.empty:
        return None

    # garantir VL_CONTA numérico; remover nulos
    if "VL_CONTA" in df.columns:
        df["VL_CONTA"] = pd.to_numeric(df["VL_CONTA"], errors="coerce")
        df = df.dropna(subset=["VL_CONTA"])
    else:
        return None

    # ajustar escala MIL → multiplicar por 1000
    if "ESCALA_MOEDA" in df.columns:
        if pd.api.types.is_categorical_dtype(df["ESCALA_MOEDA"]):
            cats = df["ESCALA_MOEDA"].cat.categories
            is_mil = df["ESCALA_MOEDA"].cat.codes == int(np.where(cats == "MIL")[0][0]) if "MIL" in cats else False
        else:
            is_mil = df["ESCALA_MOEDA"].astype(str) == "MIL"
        # aplicar ajuste com np.where (vetorizado)
        df["VL_CONTA"] = np.where(is_mil, df["VL_CONTA"] * 1000.0, df["VL_CONTA"])

    return df


# -----------------------
# Pipeline principal
# -----------------------
def parse_and_consolidate_final(
    doc_name: str,
    broad_pattern: str,
    unzipped_dir: Path = UNZIPPED_DIR,
    processed_dir: Path = PROCESSED_DIR,
) -> Dict[str, object]:
    """
    Encontra arquivos consolidados (_con_), processa, concatena, deduplica e salva parquet/csv.
    Retorna dicionário com estatísticas do processamento.
    """
    logger.info("Iniciando processamento CONSOLIDADO para: %s (padrão: %s)", doc_name.upper(), broad_pattern)
    files = find_consolidated_files(unzipped_dir, broad_pattern)
    if not files:
        logger.info("Nenhum arquivo encontrado para o padrão: %s", broad_pattern)
        return {"status": "no_files", "files_count": 0}

    df_chunks = []
    total_rows_read = 0
    total_rows_after_filter = 0
    sapr_found = False

    for path in tqdm(files, desc=f"Processando {doc_name.upper()}"):
        df = _read_csv_safe(path)
        if df is None:
            continue

        total_rows_read += len(df)

        # diagnostico SAPR (apenas relatório, sem interromper)
        if "CNPJ_CIA" in df.columns:
            df["CNPJ_CIA"] = df["CNPJ_CIA"].astype(str).str.strip()
            df_sapr = df[df["CNPJ_CIA"] == CNPJ_SAPR]
            if not df_sapr.empty and not sapr_found:
                sapr_found = True
                logger.info("[DIAGNÓSTICO SAPR11] Encontrado em: %s", path.name)
                if "ORDEM_EXERC" in df_sapr.columns:
                    logger.info("  ORDEM_EXERC values: %s", df_sapr["ORDEM_EXERC"].unique())
                if "ESCALA_MOEDA" in df_sapr.columns:
                    logger.info("  ESCALA_MOEDA values: %s", df_sapr["ESCALA_MOEDA"].unique())

        # processa e filtra o chunk
        try:
            processed = _process_df_chunk(df)
            if processed is not None and not processed.empty:
                total_rows_after_filter += len(processed)
                df_chunks.append(processed)
        except Exception as exc:
            logger.warning("Erro processando arquivo %s: %s", path, exc)
        finally:
            # liberar memória
            del df
            gc.collect()

    logger.info("Totais: linhas lidas=%d, linhas após filtros=%d", total_rows_read, total_rows_after_filter)
    if not sapr_found:
        logger.warning("[DIAGNÓSTICO SAPR11] CNPJ Sanepar (%s) NÃO encontrado na busca consolidada.", CNPJ_SAPR)

    if not df_chunks:
        logger.info("Nenhum chunk com dados válidos após filtros. Abortando concatenação.")
        return {"status": "no_data_after_filter", "files_count": len(files)}

    # concat + sort + dedupe
    logger.info("Concatenando %d chunks...", len(df_chunks))
    consolidated_df = pd.concat(df_chunks, ignore_index=True)
    consolidated_df.sort_values(by=["CNPJ_CIA", "DT_FIM_EXERC", "VERSAO"], ascending=[True, True, False], inplace=True)

    dedup_subset = ["CNPJ_CIA", "DT_FIM_EXERC", "CD_CONTA"]
    final_df = consolidated_df.drop_duplicates(subset=dedup_subset, keep="first").copy()

    # diagnóstico rápido para CNPJs de debug
    if "CNPJ_CIA" in final_df.columns:
        debug_mask = final_df["CNPJ_CIA"].astype(str).isin([str(x) for x in CNPJs_DEBUG])
        debug_data = final_df.loc[debug_mask]
        if not debug_data.empty:
            grouped = debug_data.groupby(["CNPJ_CIA", "DT_FIM_EXERC"]).size().reset_index(name="contagem_contas")
            logger.info("Dados SAPR11/VIVA3 no DF final:\n%s", grouped.head(20).to_string(index=False))
        else:
            logger.info("Nenhum dado SAPR11/VIVA3 no DF final.")

    # salvar resultados
    out_parquet = processed_dir / f"raw_{doc_name}.parquet"
    out_csv = processed_dir / f"raw_{doc_name}.csv"

    try:
        logger.info("Salvando Parquet: %s", out_parquet)
        final_df.to_parquet(out_parquet, index=False)
        logger.info("Salvando CSV: %s", out_csv)
        final_df.to_csv(out_csv, index=False, sep=";", encoding="utf-8-sig")
        logger.info("Shape final salvo: %s", final_df.shape)
    except Exception as exc:
        logger.exception("Falha ao salvar arquivos: %s", exc)
        return {"status": "save_error", "error": str(exc)}

    # limpeza final
    del df_chunks, consolidated_df, final_df
    gc.collect()

    return {
        "status": "ok",
        "files_count": len(files),
        "rows_read": total_rows_read,
        "rows_after_filter": total_rows_after_filter,
        "saved_parquet": str(out_parquet),
        "saved_csv": str(out_csv),
    }


if __name__ == "__main__":
    results = {}
    for name, pattern in DOC_PATTERNS_BROAD.items():
        results[name] = parse_and_consolidate_final(name, pattern)
        gc.collect()

    logger.info("Processo de parsing (v9.0 - Busca Consolidada) concluído!")
    logger.info("Consulte logs acima para mensagens '---> [DIAGNÓSTICO SAPR11 v9.0]'.")
    logger.info("Resumo por documento: %s", results)



---
### **🔧 cv_processor —** Transformação para formato WIDE (fundamentals_wide)

**1. Descrição curta**  
Transforma os arquivos processados (`processed/raw_dre.parquet`, `raw_bpa.parquet`, `raw_bpp.parquet`) em tabelas *wide* prontos para análise fundamentalista. Mapeia códigos de conta para nomes legíveis (plano de contas), filtra pelo ÚLTIMO exercício, pivota (long → wide) e faz o merge final entre DRE, BPA e BPP.

**2. Principais responsabilidades**
- Ler `processed/raw_{doc}.parquet` para cada doc type (dre, bpa, bpp)
- Filtrar linhas por CD_CONTA usando`MAPA_CONTAS_*
- Manter apenas ORDEM_EXERC == 'ÚLTIMO' quando disponível
- Converter VL_CONTA para numérico e remover nulos
- Pivot (index: CNPJ_CIA, DENOM_CIA, DT_FIM_EXERC; columns: nome legível da CONTA)
- Merge outer entre DRE / BPA / BPP em ['CNPJ_CIA','DENOM_CIA','DT_FIM_EXERC']
- Salvar data/cvm/final/fundamentals_wide.parquet (+ CSV ;)

**3. Entradas**
- data/cvm/processed/raw_dre.parquet
- data/cvm/processed/raw_bpa.parquet
- data/cvm/processed/raw_bpp.parquet

**4. Saídas**
- data/cvm/final/fundamentals_wide.parquet
- data/cvm/final/fundamentals_wide.csv


In [26]:
import pandas as pd
import numpy as np
from pathlib import Path
import logging
from typing import Dict, List, Optional
import gc # Para garbage collection

# -----------------------
# Configuração (ATUALIZADA)
# -----------------------
BASE_DIR = Path("data") / "cvm"
PROCESSED_DIR = BASE_DIR / "processed"
FINAL_DIR = BASE_DIR / "final"
FINAL_DIR.mkdir(parents=True, exist_ok=True)

# Mapa de contas PRINCIPAIS (agregadas)
MAPA_CONTAS_DRE_MAIN = {
    "3.01": "Receita Líquida",
    "3.02": "Custo dos Bens e/ou Serviços Vendidos",
    "3.03": "Lucro Bruto",
    "3.05": "EBIT",
    "3.07": "EBT",
    "3.11": "Lucro Líquido Consolidado",
}

MAPA_CONTAS_BPA_MAIN = {
    "1": "Ativo Total",
    "1.01": "Ativo Circulante",
    "1.02": "Ativo Não Circulante",
}

MAPA_CONTAS_BPP_MAIN = {
    "2": "Passivo Total",
    "2.01": "Passivo Circulante",
    "2.02": "Passivo Não Circulante",
    "2.03": "Patrimônio Líquido Consolidado",
}

# --- NOVO: Mapa de contas DETALHADAS (para colunas específicas) ---
# Estes códigos são comuns, mas podem precisar de ajustes finos
# Verifique o plano de contas da CVM para mais detalhes se necessário
MAPA_CONTAS_DETALHADAS_BPA = {
    "1.01.01": "Caixa e Equivalentes", # Caixa e Equivalentes de Caixa
}

MAPA_CONTAS_DETALHADAS_BPP = {
    "2.01.04": "Dívida Curto Prazo", # Empréstimos e Financiamentos CP
    "2.02.01": "Dívida Longo Prazo", # Empréstimos e Financiamentos LP
}
# --- FIM NOVO ---

# Estrutura unificada dos mapas
MAPA_CONTAS_GERAL = {
    "dre": {"main": MAPA_CONTAS_DRE_MAIN, "detailed": None}, # DRE não tem detalhado por enquanto
    "bpa": {"main": MAPA_CONTAS_BPA_MAIN, "detailed": MAPA_CONTAS_DETALHADAS_BPA},
    "bpp": {"main": MAPA_CONTAS_BPP_MAIN, "detailed": MAPA_CONTAS_DETALHADAS_BPP},
}

# Colunas chave esperadas
INDEX_COLS = ["CNPJ_CIA", "DENOM_CIA", "DT_FIM_EXERC"]

# -----------------------
# Logging
# -----------------------
logging.basicConfig(level=logging.INFO, format="%(asctime)s %(levelname)s %(message)s")
logger = logging.getLogger("cvm_transform")

# -----------------------
# Funções utilitárias
# -----------------------
def _read_processed_parquet(path: Path) -> Optional[pd.DataFrame]:
    """Lê um parquet com tratamento de erro."""
    try:
        df = pd.read_parquet(path)
        logger.info("Lido parquet: %s (shape=%s)", path.name, df.shape)
        return df
    except Exception as exc:
        logger.warning("Falha ao ler parquet %s: %s", path.name, exc)
        return None

# --- ATUALIZADO: Função process_and_pivot_file ---
def process_and_pivot_file(
    doc_name: str,
    main_account_map: Dict[str, str],
    detailed_account_map: Optional[Dict[str, str]] = None, # Aceita mapa detalhado opcional
    processed_dir: Path = PROCESSED_DIR,
) -> Optional[pd.DataFrame]:
    """
    Carrega raw parquet, filtra contas (principais E detalhadas),
    mantém 'ÚLTIMO' exercício e pivota para wide.
    """
    input_file = processed_dir / f"raw_{doc_name}.parquet"
    logger.info("Processando %s -> %s", doc_name.upper(), input_file.name)

    if not input_file.exists():
        logger.warning("Arquivo não encontrado: %s. Pulando %s.", input_file.name, doc_name)
        return None

    df = _read_processed_parquet(input_file)
    if df is None or df.empty: return None

    # Validar colunas essenciais
    essential_cols = ["CD_CONTA", "VL_CONTA", "DT_FIM_EXERC", "ORDEM_EXERC"] + INDEX_COLS
    missing_essentials = [c for c in essential_cols if c not in df.columns]
    if missing_essentials:
        logger.error("Colunas essenciais ausentes em %s: %s. Pulando.", input_file.name, missing_essentials)
        return None

    # --- LÓGICA ATUALIZADA: Combinar contas principais e detalhadas ---
    all_accounts_to_keep = list(main_account_map.keys())
    if detailed_account_map:
        all_accounts_to_keep.extend(list(detailed_account_map.keys()))
        logger.info(f"Contas detalhadas a serem extraídas: {list(detailed_account_map.keys())}")

    logger.info(f"Total de códigos de conta a serem filtrados: {len(all_accounts_to_keep)}")
    df = df[df["CD_CONTA"].isin(all_accounts_to_keep)].copy()
    logger.info("Após filtrar contas (principais + detalhadas): shape=%s", df.shape)
    if df.empty:
        logger.info("Nenhuma conta de interesse encontrada. Pulando.")
        return None
    # --- FIM LÓGICA ATUALIZADA ---

    # Converter data e filtrar por ORDEM_EXERC == 'ÚLTIMO'
    df["DT_FIM_EXERC"] = pd.to_datetime(df["DT_FIM_EXERC"], errors="coerce")
    df = df.dropna(subset=["DT_FIM_EXERC"])
    df = df[df["ORDEM_EXERC"] == "ÚLTIMO"].copy()
    logger.info("Após filtrar ORDEM_EXERC == 'ÚLTIMO': shape=%s", df.shape)
    if df.empty:
        logger.info("Nenhuma linha 'ÚLTIMO' encontrada. Pulando.")
        return None

    # --- LÓGICA ATUALIZADA: Mapear nome da conta (prioriza detalhado) ---
    # Primeiro tenta mapear com o mapa detalhado, depois com o principal
    map_detalhado = detailed_account_map if detailed_account_map else {}
    df["CONTA"] = df["CD_CONTA"].map(map_detalhado).fillna(df["CD_CONTA"].map(main_account_map))
    # Remove contas que não foram mapeadas por nenhum dos mapas (se houver)
    df = df.dropna(subset=["CONTA"])
    logger.info("Contas mapeadas. Exemplo de nomes: %s", df["CONTA"].unique()[:5])
    if df.empty:
        logger.info("Nenhuma conta mapeada resultou em nome válido. Pulando.")
        return None
    # --- FIM LÓGICA ATUALIZADA ---

    # Garantir VL_CONTA numérico e remover ausentes
    df["VL_CONTA"] = pd.to_numeric(df["VL_CONTA"], errors="coerce")
    df = df.dropna(subset=["VL_CONTA"])
    if df.empty:
        logger.info("Sem valores numéricos para VL_CONTA. Pulando.")
        return None

    # Pivot (long -> wide)
    try:
        logger.info("Pivotando (long -> wide)...")
        # Usar fill_value=0 pode ser útil se algumas contas não aparecem em todos os trimestres
        df_wide = df.pivot_table(
            index=INDEX_COLS,
            columns="CONTA", # Usará os nomes mapeados (incluindo os detalhados)
            values="VL_CONTA",
            aggfunc="sum",
            fill_value=0 # Preenche com 0 contas ausentes naquele período/empresa
        )
        df_wide = df_wide.reset_index()
        df_wide.columns.name = None # Limpa o nome do índice das colunas
        logger.info("Pivot concluído: shape=%s", df_wide.shape)
        logger.info("Colunas geradas pelo pivot: %s", df_wide.columns.tolist())
    except Exception as exc:
        logger.exception("Erro ao pivotar %s: %s", input_file.name, exc)
        return None

    del df
    gc.collect()
    return df_wide

# -----------------------
# Função para juntar todos os DF wide (dre,bpa,bpp) - SEM ALTERAÇÃO
# -----------------------
def merge_fundamentals(dfs_wide: Dict[str, pd.DataFrame]) -> Optional[pd.DataFrame]:
    """ Junta os DataFrames wide (DRE, BPA, BPP) """
    if not dfs_wide:
        logger.warning("Nenhum DataFrame wide fornecido para merge.")
        return None
    valid_dfs = {k: v for k, v in dfs_wide.items() if v is not None and not v.empty}
    if not valid_dfs:
        logger.warning("Nenhum DataFrame wide VÁLIDO fornecido para merge.")
        return None

    keys = list(valid_dfs.keys())
    base = valid_dfs[keys[0]].copy()
    logger.info("Usando %s como base para merge (shape=%s)", keys[0], base.shape)

    for k in keys[1:]:
        logger.info("Mesclando com %s (shape=%s)", k, valid_dfs[k].shape)
        # Verifica colunas duplicadas (exceto as de índice) antes do merge
        cols_to_merge = valid_dfs[k].columns.difference(base.columns).tolist() + INDEX_COLS
        base = pd.merge(base, valid_dfs[k][cols_to_merge], on=INDEX_COLS, how="outer")
        logger.info("Shape após merge com %s: %s", k, base.shape)

    base = base.sort_values(by=["CNPJ_CIA", "DT_FIM_EXERC"]).reset_index(drop=True)
    logger.info("Merge finalizado: shape=%s", base.shape)
    logger.info("Colunas finais: %s", base.columns.tolist())
    return base


def save_final(df: pd.DataFrame, final_dir: Path = FINAL_DIR, fname: str = "fundamentals_wide.parquet") -> Dict[str, str]:
    """ Salva o DataFrame final em parquet e CSV """
    final_dir.mkdir(parents=True, exist_ok=True)
    out_parquet = final_dir / fname
    out_csv = final_dir / str(fname).replace(".parquet", ".csv")

    try:
        logger.info("Salvando parquet final em: %s", out_parquet)
        df.to_parquet(out_parquet, index=False)
        logger.info("Salvando CSV final em: %s", out_csv)
        df.to_csv(out_csv, index=False, sep=";", encoding="utf-8-sig")
        return {"parquet": str(out_parquet), "csv": str(out_csv)}
    except Exception as exc:
        logger.exception("Erro ao salvar arquivo final: %s", exc)
        raise

# -----------------------
# Execução principal (ATUALIZADA)
# -----------------------
if __name__ == "__main__":
    logger.info("Iniciando transformação para formato WIDE (com contas detalhadas)...")

    dfs_wide = {}
    # --- LÓGICA ATUALIZADA: Iterar sobre a nova estrutura de mapas ---
    for doc_type, maps_dict in MAPA_CONTAS_GERAL.items():
        main_map = maps_dict.get("main")
        detailed_map = maps_dict.get("detailed") # Pode ser None

        if main_map: # Só processa se houver um mapa principal
            logger.info("-" * 20)
            df_wide = process_and_pivot_file(doc_type, main_map, detailed_map) # Passa ambos os mapas
            if df_wide is not None and not df_wide.empty:
                dfs_wide[doc_type] = df_wide
            else:
                logger.warning("Processamento de %s não gerou DataFrame válido.", doc_type)
        else:
             logger.warning("Mapa principal não definido para %s. Pulando.", doc_type)
    # --- FIM LÓGICA ATUALIZADA ---

    if not dfs_wide:
        logger.error("Nenhum dataframe produzido. Encerrando sem salvar.")
    else:
        logger.info("-" * 20)
        logger.info("Iniciando merge dos DataFrames DRE, BPA, BPP...")
        final_df = merge_fundamentals(dfs_wide)
        if final_df is None or final_df.empty:
            logger.error("DataFrame final vazio após merge. Encerrando.")
        else:
            save_paths = save_final(final_df)
            logger.info("-" * 20)
            logger.info("Processamento concluído com sucesso!")
            logger.info("Arquivos salvos: %s", save_paths)
            logger.info("Shape final mestre: %s", final_df.shape)
            logger.info("Colunas finais geradas: %s", final_df.columns.tolist())
            logger.info("Amostra do resultado final:\n%s",
                        final_df.head()[INDEX_COLS + list(final_df.columns.difference(INDEX_COLS))].to_string(index=False)) # Reordena colunas para amostra

### **Documentação:** Calculadora de Quality Score (AurumQualityScoreCalculator)

#### **1. Objetivo**

Este script é um dos pilares centrais do **Pilar 1 (Qualidade Financeira)** do projeto Aurum. Sua responsabilidade é transformar os dados fundamentalistas brutos (extraídos da CVM e formatados no `fundamentals_wide.parquet`) em métricas de performance acionáveis.

O script executa um pipeline completo que:
1.  Carrega os dados brutos da CVM.
2.  Calcula os valores **TTM (Trailing Twelve Months / Últimos 12 Meses)**, corrigindo a natureza "Acumulada no Ano" (YTD) dos dados da CVM.
3.  Calcula um conjunto abrangente de **ratios financeiros** (Rentabilidade, Margens, Alavancagem, etc.) usando os dados TTM.
4.  Calcula um **Aurum Quality Score** inicial, baseado em um *ranking percentile cross-sectional* (por data) desses ratios.
5.  Salva os resultados históricos e os mais recentes.

#### **2. Configuração (Input)**

O script depende de um único arquivo de entrada, que deve ser gerado pelo pipeline de processamento da CVM:

* **`ata/cvm/final/fundamentals_wide.parquet**: Um arquivo Parquet contendo os dados de Balanço Patrimonial (BP) e Demonstração do Resultado (DRE) em formato "wide" (uma linha por empresa/data, múltiplas colunas de métricas).
    * **Importante:** O script assume que os dados da DRE (ex: `Receita Líquida`, `Lucro Bruto`) estão no formato **YTD (Acumulado no Ano)**, que é o padrão da CVM.

#### 3. Pipeline de Execução (Passo a Passo)

A classe **AurumQualityScoreCalculator** gerencia todo o fluxo de trabalho:

##### **Passo 1:** Carregar e Preparar Dados (`load_and_prepare_data`)

* Carrega o arquivo `fundamentals_wide.parquet`.
* Chama _clean_fundamentals_data para:
    * Converter DT_FIM_EXERC para datetime.
    * Remover quaisquer linhas duplicadas por CNPJ_CIA e DT_FIM_EXERC.
    * **Ordenar** os dados por CNPJ_CIA e DT_FIM_EXERC, o que é **essencial** para o cálculo de TTM.

##### **Passo 2:** Cálculo de TTM (`calculate_ttm_data`)

Este é o passo mais crítico do script. Ele converte os dados de fluxo (DRE) de YTD para TTM.

* **Lógica:** O script agrupa por `CNPJ_CIA` e aplica a função `rolling_sum_group`.
* **Desacumulação (YTD -> Trimestral):** Para cada coluna de DRE, ele calcula o valor trimestral fazendo:
    quarterly = group[col] - group[col].shift(1).fillna(0)
* **Anualização (Trimestral -> TTM):** Em seguida, ele calcula a soma móvel dos últimos 4 valores trimestrais:
    group[f'{col}_ttm'] = quarterly.rolling(window=4, min_periods=4).sum()
* O resultado é salvo em self.ratios_df, pronto para o cálculo dos ratios.

##### Passo 3: Cálculo de Ratios Financeiros (`calculate_financial_ratios`)

* Usando o self.ratios_df (que agora contém as colunas `_ttm`), este método calcula os principais indicadores financeiros.
* **Categorias:**
    * **Contas de Dívida: Dívida Bruta, Dívida Líquida, Capital Investido**.
    * **Rentabilidade (com TTM):** ROE, ROA, ROIC.
    * **Margens (com TTM):** MARGEM_EBIT, MARGEM_LIQUIDA, MARGEM_BRUTA.
    * **Alavancagem:** ALAVANCAGEM (Passivo/Ativo), DIVIDA_PL, DIVIDA_LIQ_EBIT.
    * **Liquidez:** LIQUIDEZ_CORRENTE.
    * **Eficiência (com TTM):** GIRO_ATIVO.
* Usa safe_divide para evitar erros de divisão por zero.

##### Passo 3.1: Tratamento de Outliers (`_handle_ratio_outliers`)

* Após o cálculo, os ratios são limpos.
* Valores `infinitos` são substituídos por `NaN`.
* Os dados são "winsorizados": valores extremos são "clipados" (limitados) aos **percentis 1% (inferior) e 99% (superior)**. Isso torna o scoring subsequente mais robusto.

##### Passo 4: Cálculo do Score de Qualidade (`calculate_quality_scores`)

Este método implementa a **Metodologia de Ranking Cross-Sectional**.

* **Configuração:** Define as métricas, pesos e a direção (ex: `ROIC` -> maior é melhor, `DIVIDA_LIQ_EBIT` -> menor é melhor).
* **Ranking por Data:** O script agrupa os dados por `DT_FIM_EXERC` (`grouped_by_date`).
* Para cada métrica, ele calcula o **ranking percentile** de cada empresa *dentro daquele período*:
    `scores_df[score_col] = grouped_by_date[metric].rank(pct=True) * 100`
* **Score Composto:** O `aurum_quality_score` é calculado como a soma ponderada desses rankings (percentis de 0 a 100). Valores `NaN` são preenchidos com a mediana (50) para não penalizar empresas excessivamente.
* **Classificação Final:** O script calcula `quality_quintile` e `quality_grade` (A-E) com base no ranking *final* do `aurum_quality_score` de cada período.

##### Passo 5 e 6: Salvar Resultados (`get_latest_scores` e `save_results`)

* O script gera e salva quatro arquivos distintos no diretório `data/aurum_scores_output/`.

#### 6. Saída (Output)

A execução do script gera os seguintes arquivos em `data/aurum_scores_output/`:

1.  **`aurum_quality_scores_complete.parquet`**:
    * O arquivo mais importante. Contém o **histórico completo** de todas as empresas, datas, ratios calculados e scores. Este será o input para o `AurumScoringSystem` avançado e para o *backtesting*.

2.  **`aurum_quality_scores_latest.parquet`**:
    * Um arquivo de conveniência que contém apenas o **último registro (data mais recente)** de score/ratios para cada empresa.

3.  **`aurum_quality_scores_latest.csv`**:
    * Versão CSV do arquivo acima, para fácil visualização em planilhas.

4.  **`aurum_scores_statistics.txt`**:
    * Um relatório de texto (`.txt`) legível, mostrando as estatísticas do último período (distribuição de notas, Top 10 empresas) para uma verificação rápida.

---

In [28]:
import pandas as pd
import numpy as np
from pathlib import Path
import logging
from typing import Dict, List, Optional
import warnings
from tqdm import tqdm

warnings.filterwarnings('ignore')
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)
tqdm.pandas()

class AurumQualityScoreCalculator:
    """ Calcula TTM, ratios financeiros (incluindo ROIC correto) e scores """

    def __init__(self, fundamentals_path: str):
        self.fundamentals_path = Path(fundamentals_path)
        self.fundamentals_df = None
        self.ratios_df = None
        self.scores_df = None
        self.dre_cols = [
            'Receita Líquida', 'Custo dos Bens e/ou Serviços Vendidos',
            'Lucro Bruto', 'EBIT', 'EBT', 'Lucro Líquido Consolidado'
        ]

    def load_and_prepare_data(self) -> pd.DataFrame:
        logger.info("📥 1. CARREGANDO NOVO fundamentals_wide.parquet...")
        try:
            self.fundamentals_df = pd.read_parquet(self.fundamentals_path)
            logger.info(f"✅ Dados carregados: {self.fundamentals_df.shape}")
            logger.info(f"Colunas encontradas: {self.fundamentals_df.columns.tolist()}")
            # Verificar se as novas colunas estão presentes
            new_cols = ['Caixa e Equivalentes', 'Dívida Curto Prazo', 'Dívida Longo Prazo']
            missing_new = [col for col in new_cols if col not in self.fundamentals_df.columns]
            if missing_new:
                 logger.error(f"❌ ERRO: Novas colunas {missing_new} NÃO encontradas no input!")
                 raise ValueError(f"Novas colunas faltando: {missing_new}")
            else:
                 logger.info("✅ Novas colunas (Caixa, Dívida CP, Dívida LP) encontradas!")

            self.fundamentals_df = self._clean_fundamentals_data(self.fundamentals_df)
            return self.fundamentals_df
        except Exception as e:
            logger.error(f"❌ Erro ao carregar dados: {e}")
            raise

    def _clean_fundamentals_data(self, df: pd.DataFrame) -> pd.DataFrame:
        df_clean = df.copy()
        df_clean['DT_FIM_EXERC'] = pd.to_datetime(df_clean['DT_FIM_EXERC'], errors='coerce')
        df_clean = df_clean.dropna(subset=['DT_FIM_EXERC'])
        df_clean = df_clean.drop_duplicates(subset=['CNPJ_CIA', 'DT_FIM_EXERC'])
        df_clean = df_clean.sort_values(['CNPJ_CIA', 'DT_FIM_EXERC'])
        return df_clean

    def calculate_ttm_data(self) -> pd.DataFrame:
        if self.fundamentals_df is None: raise ValueError("Dados não carregados.")
        logger.info("⏳ 2. CALCULANDO TTM...")
        df_ttm = self.fundamentals_df.copy()
        def rolling_sum_group(group):
            for col in self.dre_cols:
                if col in group.columns:
                    quarterly = group[col] - group[col].shift(1).fillna(0)
                    group[f'{col}_ttm'] = quarterly.rolling(window=4, min_periods=4).sum()
                else: group[f'{col}_ttm'] = np.nan
            return group
        grouped = df_ttm.groupby('CNPJ_CIA', group_keys=False)
        self.ratios_df = grouped.progress_apply(rolling_sum_group)
        logger.info(f"✅ TTM calculado.")
        return self.ratios_df

    def calculate_financial_ratios(self) -> pd.DataFrame:
        """ PASSO 3: Calcula ratios (com ROIC correto e ratios de dívida) """
        if self.ratios_df is None: raise ValueError("Dados TTM não calculados.")
        logger.info("🧮 3. CALCULANDO RATIOS FINANCEIROS (COM TTM)...")
        df = self.ratios_df.copy()

        def get_col(df, col_name): return df.get(col_name, np.nan)
        def safe_divide(num, den): return np.where(den == 0, np.nan, num / den)

        # --- Contas de Dívida (USANDO AS NOVAS COLUNAS) ---
        divida_cp = get_col(df, 'Dívida Curto Prazo').fillna(0) # Usando a coluna correta
        divida_lp = get_col(df, 'Dívida Longo Prazo').fillna(0) # Usando a coluna correta
        caixa = get_col(df, 'Caixa e Equivalentes').fillna(0)   # Usando a coluna correta
        pl = get_col(df, 'Patrimônio Líquido Consolidado').fillna(0)
        
        df['Dívida Bruta'] = divida_cp + divida_lp
        df['Capital Investido'] = df['Dívida Bruta'] + pl # Definição correta
        df['Dívida Líquida'] = df['Dívida Bruta'] - caixa # Definição correta
        logger.info("✅ Dívida Bruta, Capital Investido e Dívida Líquida calculados.")

        # --- RENTABILIDADE ---
        logger.info("📈 Calculando RENTABILIDADE...")
        df['ROE'] = safe_divide(get_col(df, 'Lucro Líquido Consolidado_ttm'), pl)
        df['ROA'] = safe_divide(get_col(df, 'Lucro Líquido Consolidado_ttm'), get_col(df, 'Ativo Total'))
        # <<< ROIC CORRETO >>>
        df['ROIC'] = safe_divide(get_col(df, 'EBIT_ttm'), df['Capital Investido'])
        logger.info(f"✅ ROIC (correto) calculado. Média: {df['ROIC'].mean():.4f}")
        # <<< FIM ROIC CORRETO >>>

        # --- MARGENS ---
        df['MARGEM_EBIT'] = safe_divide(get_col(df, 'EBIT_ttm'), get_col(df, 'Receita Líquida_ttm'))
        df['MARGEM_LIQUIDA'] = safe_divide(get_col(df, 'Lucro Líquido Consolidado_ttm'), get_col(df, 'Receita Líquida_ttm'))
        df['MARGEM_BRUTA'] = safe_divide(get_col(df, 'Lucro Bruto_ttm'), get_col(df, 'Receita Líquida_ttm'))

        # --- ALAVANCAGEM ---
        logger.info("⚖️ Calculando ALAVANCAGEM...")
        df['ALAVANCAGEM'] = safe_divide(get_col(df, 'Passivo Total'), get_col(df, 'Ativo Total'))
        df['DIVIDA_PL'] = safe_divide(df['Dívida Bruta'], pl) # Agora pode ser calculado
        df['DIVIDA_LIQ_EBIT'] = safe_divide(df['Dívida Líquida'], get_col(df, 'EBIT_ttm')) # Agora pode ser calculado
        logger.info("✅ Ratios de alavancagem (incluindo DIVIDA_PL, DIVIDA_LIQ_EBIT) calculados.")

        # --- LIQUIDEZ ---
        df['LIQUIDEZ_CORRENTE'] = safe_divide(get_col(df, 'Ativo Circulante'), get_col(df, 'Passivo Circulante'))

        # --- EFICIÊNCIA ---
        df['GIRO_ATIVO'] = safe_divide(get_col(df, 'Receita Líquida_ttm'), get_col(df, 'Ativo Total'))

        self.ratios_df = self._handle_ratio_outliers(df)
        logger.info("🎯 RATIOS CALCULADOS e limpos.")
        logger.info(f"Colunas FINAIS de Ratios: {self.ratios_df.columns.tolist()}")
        return self.ratios_df

    def _handle_ratio_outliers(self, df: pd.DataFrame) -> pd.DataFrame:
        df_clean = df.copy()
        # Lista completa agora que temos os dados
        ratio_columns = [
            'ROE', 'ROA', 'ROIC', 'MARGEM_EBIT', 'MARGEM_LIQUIDA', 'MARGEM_BRUTA',
            'ALAVANCAGEM', 'DIVIDA_PL', 'DIVIDA_LIQ_EBIT', 'LIQUIDEZ_CORRENTE', 'GIRO_ATIVO'
        ]
        for col in ratio_columns:
            if col in df_clean.columns:
                df_clean[col] = df_clean[col].replace([np.inf, -np.inf], np.nan)
                if df_clean[col].notna().sum() > 0:
                    lower = df_clean[col].quantile(0.01)
                    upper = df_clean[col].quantile(0.99)
                    df_clean[col] = df_clean[col].clip(lower=lower, upper=upper)
        return df_clean

    def calculate_quality_scores(self) -> pd.DataFrame:
        """ PASSO 4: Calcula scores individuais e score composto """
        if self.ratios_df is None: raise ValueError("Ratios não calculados.")
        logger.info("🎯 4. CALCULANDO SCORES DE QUALIDADE...")
        scores_df = self.ratios_df.copy()
        grouped_by_date = scores_df.groupby('DT_FIM_EXERC')

        # Configuração original (ou ajuste como preferir)
        metrics_config = {
            'ROIC': {'direction': 1, 'weight': 0.25}, # ROIC correto
            'ROE': {'direction': 1, 'weight': 0.15},
            'MARGEM_EBIT': {'direction': 1, 'weight': 0.15},
            'MARGEM_LIQUIDA': {'direction': 1, 'weight': 0.10},
            'DIVIDA_LIQ_EBIT': {'direction': -1, 'weight': 0.15}, # Agora existe
            'LIQUIDEZ_CORRENTE': {'direction': 1, 'weight': 0.10},
            'GIRO_ATIVO': {'direction': 1, 'weight': 0.10},
            # 'ALAVANCAGEM': {'direction': -1, 'weight': 0.0}, # Pode remover ou ajustar peso
        }
        # Validar e ajustar pesos para somar 1.0
        total_w = sum(c['weight'] for c in metrics_config.values())
        if abs(total_w - 1.0) > 0.01:
             logger.warning(f"Soma dos pesos é {total_w:.2f}. Ajustando proporcionalmente...")
             for k in metrics_config: metrics_config[k]['weight'] /= total_w

        logger.info(f"Configuração de métricas para score: { {k: v['weight'] for k, v in metrics_config.items()} }")

        for metric, config in metrics_config.items():
            if metric in scores_df.columns and scores_df[metric].notna().any():
                score_col = f'score_{metric}'
                if config['direction'] == 1:
                    scores_df[score_col] = grouped_by_date[metric].rank(pct=True) * 100
                else:
                    scores_df[score_col] = grouped_by_date[metric].rank(ascending=False, pct=True) * 100
                logger.info(f"  ✅ Score {metric} calculado.")
            else:
                 logger.warning(f"Métrica '{metric}' não encontrada ou sem dados para score.")

        logger.info("⚖️ Calculando score composto...")
        scores_df['aurum_quality_score'] = 0.0
        total_applied_weight = 0.0
        for metric, config in metrics_config.items():
            score_col = f'score_{metric}'
            if score_col in scores_df.columns:
                scores_df['aurum_quality_score'] += scores_df[score_col].fillna(50) * config['weight']
                total_applied_weight += config['weight']

        if total_applied_weight > 0:
            scores_df['aurum_quality_score'] /= total_applied_weight

        # ... (código de classificação e restante igual ao anterior) ...
        logger.info("🏆 Classificando empresas...")
        try:
             valid_scores = scores_df['aurum_quality_score'].dropna()
             if not valid_scores.empty:
                  ranks = valid_scores.rank(ascending=False, pct=True)
                  scores_df['final_rank'] = ranks
                  try:
                       scores_df['quality_quintile'] = pd.qcut(scores_df['final_rank'].dropna(), 5, labels=[f'{i}º Quintil' for i in range(1, 6)])
                  except ValueError: scores_df['quality_quintile'] = pd.cut(scores_df['final_rank'].dropna(), 5, labels=False)
                  scores_df['quality_grade'] = pd.cut(scores_df['final_rank'].dropna(), bins=[0, 0.2, 0.4, 0.6, 0.8, 1.0], labels=['A', 'B', 'C', 'D', 'E'], right=True, include_lowest=True)
             else: scores_df[['final_rank', 'quality_quintile', 'quality_grade']] = np.nan
        except Exception as e:
             logger.error(f"Erro ao calcular ranks/grades: {e}")
             scores_df[['final_rank', 'quality_quintile', 'quality_grade']] = np.nan
        
        self.scores_df = scores_df
        logger.info(f"Colunas FINAIS ANTES de salvar: {self.scores_df.columns.tolist()}")
        return scores_df

    # Métodos get_latest_scores, save_results, _save_statistics são iguais aos anteriores
    # ... (Copie e cole os métodos get_latest_scores, save_results, _save_statistics da versão anterior aqui) ...
    def get_latest_scores(self) -> pd.DataFrame:
        if self.scores_df is None: raise ValueError("Scores não calculados.")
        latest_scores = self.scores_df.sort_values('DT_FIM_EXERC').groupby('CNPJ_CIA').last().reset_index()
        return latest_scores

    def save_results(self, output_dir: str = "data/aurum_scores"):
        output_path = Path(output_dir)
        output_path.mkdir(parents=True, exist_ok=True)
        if self.scores_df is not None:
            logger.info(f"Salvando DataFrame com colunas: {self.scores_df.columns.tolist()}")
            if 'ROIC' not in self.scores_df.columns: logger.error("ERRO FATAL: Coluna 'ROIC' AUSENTE antes de salvar!")
            scores_path = output_path / "aurum_quality_scores_complete.parquet"
            self.scores_df.to_parquet(scores_path, index=False)
            logger.info(f"💾 Scores completos (histórico) salvos: {scores_path}")
        latest_scores = self.get_latest_scores()
        latest_path = output_path / "aurum_quality_scores_latest.parquet"
        latest_csv_path = output_path / "aurum_quality_scores_latest.csv"
        latest_scores.to_parquet(latest_path, index=False)
        latest_scores.to_csv(latest_csv_path, index=False, sep=';', encoding='utf-8-sig')
        logger.info(f"💾 Scores mais recentes salvos: {latest_path} e {latest_csv_path}")
        stats_path = output_path / "aurum_scores_statistics.txt"
        self._save_statistics(latest_scores, stats_path)
        logger.info(f"💾 Estatísticas salvas: {stats_path}")
        return {'complete_scores': scores_path,'latest_scores': latest_path,'statistics': stats_path}

    def _save_statistics(self, scores_df: pd.DataFrame, stats_path: Path):
        with open(stats_path, 'w', encoding='utf-8') as f:
            f.write("AURUM QUALITY SCORE - ESTATÍSTICAS (ÚLTIMO PERÍODO)\n" + "=" * 50 + "\n\n")
            f.write(f"Total de empresas: {len(scores_df)}\n")
            max_date = scores_df['DT_FIM_EXERC'].max()
            f.write(f"Período mais recente: {max_date if pd.notna(max_date) else 'N/A'}\n\n")
            if 'aurum_quality_score' in scores_df.columns:
                 f.write("DISTRIBUIÇÃO DOS SCORES:\n" + f"  Média: {scores_df['aurum_quality_score'].mean():.2f}\n" +
                         f"  Mediana: {scores_df['aurum_quality_score'].median():.2f}\n" + f"  Mínimo: {scores_df['aurum_quality_score'].min():.2f}\n" +
                         f"  Máximo: {scores_df['aurum_quality_score'].max():.2f}\n\n")
            else: f.write("DISTRIBUIÇÃO DOS SCORES: N/A\n\n")
            if 'quality_grade' in scores_df.columns:
                 f.write("DISTRIBUIÇÃO POR NOTAS:\n")
                 grade_counts = scores_df['quality_grade'].value_counts().sort_index(ascending=True)
                 for grade, count in grade_counts.items(): f.write(f"  Nota {grade}: {count} empresas\n")
            else: f.write("DISTRIBUIÇÃO POR NOTAS: N/A\n")
            if 'aurum_quality_score' in scores_df.columns:
                 f.write("\nTOP 10 EMPRESAS:\n")
                 top_10 = scores_df.nlargest(10, 'aurum_quality_score')[['DENOM_CIA', 'aurum_quality_score', 'quality_grade']]
                 for i, (_, row) in enumerate(top_10.iterrows(), 1):
                      grade = row.get('quality_grade', 'N/A')
                      f.write(f"  {i:2d}. {str(row['DENOM_CIA'])[:35]:35} {row['aurum_quality_score']:6.2f} (Nota {grade})\n")
            else: f.write("\nTOP 10 EMPRESAS: N/A\n")


# ==================== EXECUÇÃO PRINCIPAL ====================
def main():
    logger.info("🚀 INICIANDO PIPELINE DO AURUM QUALITY SCORE (Versão Final com ROIC Correto)")
    try:
        calculator = AurumQualityScoreCalculator(
            fundamentals_path="data/cvm/final/fundamentals_wide.parquet" # Ler o NOVO input
        )
        calculator.load_and_prepare_data()
        calculator.calculate_ttm_data()
        calculator.calculate_financial_ratios() # Calcular ROIC correto e ratios de dívida
        calculator.calculate_quality_scores()   # Usar ROIC correto e ratios de dívida
        output_files = calculator.save_results(output_dir="data/aurum_scores") # Salvar no diretório correto
        logger.info(f"💾 Resultados (com ROIC correto) salvos em: {output_files}")

        latest_scores = calculator.get_latest_scores()
        # ... (impressão dos resultados igual) ...
        print("\n" + "="*60 + "\n🎉 AURUM QUALITY SCORE - RESULTADOS FINAIS (ÚLTIMO PERÍODO)\n" + "="*60)
        print(f"\n📊 TOTAL DE EMPRESAS: {len(latest_scores)}")
        if 'aurum_quality_score' in latest_scores.columns: print(f"📈 SCORE MÉDIO: {latest_scores['aurum_quality_score'].mean():.2f}")
        print(f"\n📋 DISTRIBUIÇÃO DAS NOTAS:")
        if 'quality_grade' in latest_scores.columns:
             grade_dist = latest_scores['quality_grade'].value_counts().sort_index(ascending=True)
             for grade, count in grade_dist.items(): print(f"   Nota {grade}: {count:3d} empresas")
        print(f"\n🥇 TOP 10 EMPRESAS:")
        if 'aurum_quality_score' in latest_scores.columns:
             top_10 = latest_scores.nlargest(10, 'aurum_quality_score')[['DENOM_CIA', 'aurum_quality_score', 'quality_grade']]
             for i, (_, row) in enumerate(top_10.iterrows(), 1):
                  grade = row.get('quality_grade', 'N/A')
                  print(f"   {i:2d}. {str(row['DENOM_CIA'])[:35]:35} {row['aurum_quality_score']:6.2f} (Nota {grade})")

        return calculator

    except Exception as e:
        logger.error(f"❌ ERRO NO PIPELINE: {e}")
        import traceback
        logger.error(traceback.format_exc())
        raise

if __name__ == "__main__":
    aurum_calculator = main()

100%|██████████| 727/727 [00:06<00:00, 106.92it/s]



🎉 AURUM QUALITY SCORE - RESULTADOS FINAIS (ÚLTIMO PERÍODO)

📊 TOTAL DE EMPRESAS: 727
📈 SCORE MÉDIO: 49.33

📋 DISTRIBUIÇÃO DAS NOTAS:
   Nota A: 201 empresas
   Nota B:  82 empresas
   Nota C:  72 empresas
   Nota D: 146 empresas
   Nota E: 226 empresas

🥇 TOP 10 EMPRESAS:
    1. CAMIL ALIMENTOS S.A.                100.00 (Nota A)
    2. MINUPAR PARTICIPACOES S.A.           91.13 (Nota A)
    3. 521 PARTICIPACOES S.A. - EM LIQUIDA  89.17 (Nota A)
    4. STEIN SP II PARTICIPAÇÕES S.A.       87.37 (Nota A)
    5. EPR INFRAESTRUTURA PR S.A.           85.60 (Nota A)
    6. COMERCIAL QUINTELLA COM EXP SA EM L  84.67 (Nota A)
    7. SONDOTECNICA ENGENHARIA SOLOS S.A.   84.42 (Nota A)
    8. MÉLIUZ S.A.                          83.91 (Nota A)
    9. FICTOR ALIMENTOS S.A.                82.46 (Nota A)
   10. GRANJA FARIA S.A.                    82.18 (Nota A)


In [ ]:
import pandas as pd
import numpy as np
import os
import logging
from pathlib import Path

# --- Configuração ---
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# --- Arquivos de Input (VERIFICADOS) ---
PATH_PRECOS_LIMPOS = "data/historical/all_histories_cleaned.parquet"
PATH_PRECOS_WIDE = "data/historical/prices_close_wide.parquet"
PATH_FUNDAMENTOS = "data/aurum_scores/aurum_quality_scores_complete.parquet" # Seu arquivo de fundamentos com ROIC
PATH_SENTIMENTO = "data/news/news_with_sentiment.parquet"
PATH_DE_PARA = "data/ticker_cnpj_map.parquet" # Seu arquivo de mapeamento

# --- Arquivo de Output ---
OUTPUT_DIR = "data"
OUTPUT_FILENAME = "aurum_master_features.parquet"
output_path = Path(OUTPUT_DIR) / OUTPUT_FILENAME

def calcular_volatilidade_mensal():
    # ... (código igual, sem alterações) ...
    logger.info("Iniciando Unificação (Passo 1/3): Cálculo da Volatilidade...")
    try: df_prices_daily = pd.read_parquet(PATH_PRECOS_LIMPOS)
    except FileNotFoundError: logger.error(f"Arquivo não encontrado: {PATH_PRECOS_LIMPOS}"); return None
    df_prices_daily['date'] = pd.to_datetime(df_prices_daily['date'])
    df_prices_daily['returns'] = df_prices_daily.groupby('ticker')['Adj Close'].pct_change()
    df_prices_daily['VOLATILIDADE'] = df_prices_daily.groupby('ticker')['returns'].rolling(window=63, min_periods=30).std().reset_index(0, drop=True)
    df_vol_mensal = df_prices_daily.set_index('date').groupby('ticker').resample('M').last()['VOLATILIDADE'].reset_index()
    logger.info(f"✅ Volatilidade mensal calculada.")
    return df_vol_mensal

def agregar_sentimento_mensal():
    # ... (código igual, sem alterações) ...
    logger.info("Iniciando Unificação (Passo 2/3): Agregação de Sentimento...")
    try: df_sent_raw = pd.read_parquet(PATH_SENTIMENTO)
    except FileNotFoundError: logger.error(f"Arquivo não encontrado: {PATH_SENTIMENTO}"); return None
    df_sent_raw = df_sent_raw.rename(columns={'ticker_query': 'ticker', 'published_date': 'date'})
    df_sent_raw['date'] = pd.to_datetime(df_sent_raw['date'], utc=True).dt.tz_localize(None)
    if not df_sent_raw['ticker'].str.contains('.SA').any():
        logger.warning("Tickers sem sufixo .SA. Adicionando...")
        df_sent_raw['ticker'] = df_sent_raw['ticker'].apply(lambda x: f"{x}.SA" if not str(x).endswith(".SA") else x)
    df_sent_mensal = df_sent_raw.set_index('date').groupby('ticker').resample('M').agg(
        SENTIMENT_MEDIO=('numeric_sentiment', 'mean'),
        SENTIMENT_STD=('numeric_sentiment', 'std'),
        NEWS_COUNT=('ticker', 'count')
    ).reset_index()
    logger.info(f"✅ Sentimento mensal agregado.")
    return df_sent_mensal

def unificar_dataframe_mestre(df_vol_mensal, df_sent_mensal):
    if df_vol_mensal is None or df_sent_mensal is None: return
    logger.info("Iniciando Unificação (Passo 3/3): Junção do DataFrame Mestre...")

    # --- 1. Carregar Base de Preços Mensal ---
    try:
        df_close_wide = pd.read_parquet(PATH_PRECOS_WIDE)
        df_base_mensal = df_close_wide.melt(ignore_index=False, var_name='ticker', value_name='Adj Close').reset_index()
        df_base_mensal = df_base_mensal.rename(columns={'index': 'date'})
        logger.info(f"Base de preços (wide) carregada.")
    except FileNotFoundError: logger.error(f"Arquivo não encontrado: {PATH_PRECOS_WIDE}"); return

    # --- 2. Carregar Fundamentos (Trimestrais) ---
    try:
        df_fund = pd.read_parquet(PATH_FUNDAMENTOS)
        df_fund = df_fund.rename(columns={'DT_FIM_EXERC': 'date'})
        df_fund['date'] = pd.to_datetime(df_fund['date'])
        logger.info(f"Fundamentos carregados. Colunas: {df_fund.columns.tolist()}")
    except FileNotFoundError: logger.error(f"Arquivo não encontrado: {PATH_FUNDAMENTOS}"); return
    except KeyError as e: logger.error(f"Erro ao renomear 'DT_FIM_EXERC': {e}"); return

    # --- 3. Carregar o Mapeamento (DE-PARA) ---
    try:
        df_mapping = pd.read_parquet(PATH_DE_PARA)
        if 'ticker' not in df_mapping.columns or 'CNPJ_CIA' not in df_mapping.columns:
             logger.error(f"ERRO: Mapeamento {PATH_DE_PARA} sem 'ticker' ou 'CNPJ_CIA'.")
             return
    except FileNotFoundError: logger.error(f"Arquivo não encontrado: {PATH_DE_PARA}"); return

    df_fund_com_ticker = pd.merge(df_fund, df_mapping, on='CNPJ_CIA', how='left')
    df_fund_com_ticker = df_fund_com_ticker.dropna(subset=['ticker'])
    logger.info("Fundamentos mapeados para tickers.")

    # --- 4. Construir o DataFrame Mestre ---
    df_master = df_base_mensal.sort_values(by='date')
    df_fund_com_ticker = df_fund_com_ticker.sort_values(by='date')

    # 4a. Juntar Fundamentos (merge_asof)
    df_master = pd.merge_asof(
        df_master, df_fund_com_ticker, on='date', by='ticker', direction='backward'
    )
    logger.info("Merge 'as-of' dos fundamentos concluído.")
    logger.info(f"Colunas no df_master APÓS merge_asof: {df_master.columns.tolist()}") # DEBUG

    # 4b. Juntar Volatilidade (merge)
    df_master = pd.merge(df_master, df_vol_mensal, on=['date', 'ticker'], how='left')
    logger.info("Merge da volatilidade concluído.")

    # 4c. Juntar Sentimento (merge)
    df_master = pd.merge(df_master, df_sent_mensal, on=['date', 'ticker'], how='left')
    logger.info("Merge do sentimento concluído.")

    # --- 5. Limpeza Final ---
    df_master['SENTIMENT_MEDIO'] = df_master['SENTIMENT_MEDIO'].fillna(0)
    df_master['VOLATILIDADE'] = df_master.groupby('ticker')['VOLATILIDADE'].ffill().bfill()

    # <<< VERSÃO FINAL DO dropna >>>
    # Usando ambas as colunas, pois df_master.info() provou que elas existem
    fundamental_key_columns = ['ROE', 'ROIC'] # <--- USANDO AMBAS AS COLUNAS

    missing_cols = [col for col in fundamental_key_columns if col not in df_master.columns]
    if missing_cols:
        # Este erro não deve mais acontecer baseado no seu df.info()
        logger.error(f"ERRO CRÍTICO PÓS-MERGE: Colunas {missing_cols} não encontradas!")
        return
    else:
        logger.info(f"Aplicando dropna nas colunas chave: {fundamental_key_columns}")
        df_master = df_master.dropna(subset=fundamental_key_columns)
    # <<< FIM DA VERSÃO FINAL >>>

    logger.info(f"Limpeza final concluída. DataFrame Mestre pronto com {len(df_master)} linhas.")

    # --- 6. Salvar o Novo DataFrame Mestre ---
    os.makedirs(OUTPUT_DIR, exist_ok=True)
    df_master.to_parquet(output_path, index=False)
    logger.info(f"✅✅✅ DataFrame Mestre salvo em: {output_path} ✅✅✅")

    print("\n--- Informações do DataFrame Mestre Gerado ---")
    df_master.info(verbose=True, show_counts=True) # Mostrar detalhes
    print("\n--- Amostra do DataFrame Mestre ---")
    sample_cols = ['date', 'ticker', 'ROE', 'ROIC', 'SENTIMENT_MEDIO', 'VOLATILIDADE']
    print(df_master.sample(5)[[col for col in sample_cols if col in df_master.columns]])

    return df_master

# --- Execução Principal ---
if __name__ == "__main__":
    df_vol = calcular_volatilidade_mensal()
    df_sent = agregar_sentimento_mensal()
    if df_vol is not None and df_sent is not None:
        unificar_dataframe_mestre(df_vol, df_sent)
    else:
        logger.error("Falha ao gerar dados. O DataFrame Mestre não foi criado.")


--- Informações do DataFrame Mestre Gerado ---
<class 'pandas.core.frame.DataFrame'>
Index: 19581 entries, 2232 to 34531
Data columns (total 57 columns):
 #   Column                                     Non-Null Count  Dtype         
---  ------                                     --------------  -----         
 0   date                                       19581 non-null  datetime64[ns]
 1   ticker                                     19581 non-null  object        
 2   Adj Close                                  19581 non-null  float64       
 3   CNPJ_CIA                                   19581 non-null  object        
 4   DENOM_CIA                                  19581 non-null  object        
 5   Custo dos Bens e/ou Serviços Vendidos      19581 non-null  float64       
 6   EBIT                                       19581 non-null  float64       
 7   EBT                                        19581 non-null  float64       
 8   Lucro Bruto                                19581 n

### **Documentação:** Sistema de Scoring Avançado (AurumScoringSystem)

#### 1. Objetivo

Este script é o "cérebro" do projeto Aurum, onde a tese de investimento quantitativo é de fato implementada. Ele representa a **evolução** do `AurumQualityScoreCalculator`, aplicando uma metodologia de scoring mais robusta e academicamente embasada.

Enquanto o script anterior (`Calculator`) era focado em *calcular ratios* e criar um *score de ranking simples*, este script (`System`) tem como responsabilidades:
1.  **Carregar uma Tese:** Define um conjunto explícito de **métricas e pesos** baseados em teorias financeiras (ex: Fama & French, Graham & Dodd), armazenados na classe `ScoringMetric`.
2.  **Normalização Avançada:** Substitui a normalização por ranking (percentil) por uma **normalização sigmóide (baseada em Z-Score)**. Isso cria um score mais suave, robusto a outliers e que recompensa melhor empresas excepcionais.
3.  **Incorporar Fatores Múltiplos:** O sistema é projetado para consumir *não apenas* os ratios financeiros do script anterior, mas também métricas de **crescimento**, **sentimento (NLP)** e **volatilidade (preço)**.
4.  **Validar-se:** Gera um relatório de validação que compara a contribuição *teórica* de cada métrica (o peso que definimos) com sua contribuição *real* no score final.
5.  **Gerar o Score Final:** Calcula o `aurum_quality_score` final e as classificações (A, B, C, D, E).

#### 2. Configuração (Input)

Este script é o **segundo passo** no pipeline de scoring e depende da saída do script anterior.

1.  **Input Principal (Obrigatório):**
    * `data/aurum_scores/aurum_quality_scores_complete.parquet`: Este é o arquivo de **output** gerado pelo `AurumQualityScoreCalculator`. Ele contém todos os ratios históricos (ROE, ROIC, etc.) já calculados e tratados.

2.  **Input Opcional (Configuração):**
    * O construtor `AurumScoringSystem(config_path="...")` aceita um caminho para um arquivo `.json`. Isso permite carregar um conjunto personalizado de métricas e pesos sem alterar o código, facilitando a experimentação. Se nenhum caminho for fornecido, ele usa os pesos padrão definidos em `_initialize_scoring_metrics`.

#### 3. Saída (Output)

O script cria um novo diretório: `data/aurum_final_scores/`.

1.  **`aurum_advanced_scores.parquet`**:
    * O **histórico completo** de todas as empresas com os scores finais (0-100), notas (A-E) e quintis. Este é o arquivo final que será usado para o **backtesting**.

2.  **`aurum_latest_advanced_scores.parquet`**:
    * Um snapshot contendo apenas o **último score disponível** para cada empresa.

3.  **`aurum_scoring_config.json`**:
    * Um arquivo `.json` que salva a tese exata (métricas e pesos) usada nesta execução, garantindo a reprodutibilidade dos resultados.

In [13]:
import pandas as pd
import numpy as np
from pathlib import Path
import logging
from typing import Dict, List, Optional, Tuple
import warnings
from dataclasses import dataclass
import json

warnings.filterwarnings('ignore')

# Configuração de logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

@dataclass
class ScoringMetric:
    """Classe para representar uma métrica de scoring"""
    name: str
    weight: float
    direction: int  # 1 = maior é melhor, -1 = menor é melhor
    description: str
    min_value: float = None
    max_value: float = None
    ideal_range: Tuple[float, float] = None

class AurumScoringSystem:
    """
    Sistema avançado de scoring para o Aurum Quality Score
    com pesos baseados em fundamentos financeiros
    """
    
    def __init__(self, config_path: str = None):
        self.scoring_metrics = self._initialize_scoring_metrics(config_path)
        self.quality_thresholds = {
            'A': 80,  # Excelente
            'B': 60,  # Bom
            'C': 40,  # Regular
            'D': 20,  # Ruim
            'E': 0    # Muito ruim
        }
        
    def _initialize_scoring_metrics(self, config_path: str = None) -> Dict[str, ScoringMetric]:
        """
        Inicializa as métricas de scoring com pesos baseados em:
        - Graham & Dodd: Security Analysis
        - Fama & French: Three Factor Model  
        - Práticas do mercado quantitativo
        """
        
        if config_path and Path(config_path).exists():
            return self._load_custom_config(config_path)
        
        # Pesos baseados em importância relativa para qualidade de empresas
        # CORREÇÃO: Soma total = 1.0 (100%)
        metrics_config = {
            # === RENTABILIDADE (47% do total) ===
            'ROE': ScoringMetric(
                name='ROE', weight=0.18, direction=1,  # Aumentado de 0.15 para 0.18
                description='Return on Equity - Eficiência do capital próprio',
                ideal_range=(0.10, 0.25)
            ),
            'ROA': ScoringMetric(
                name='ROA', weight=0.12, direction=1,
                description='Return on Assets - Eficiência dos ativos',
                ideal_range=(0.05, 0.15)
            ),
            'MARGEM_EBIT': ScoringMetric(
                name='MARGEM_EBIT', weight=0.08, direction=1,
                description='Margem Operacional - Ebit/Receita',
                ideal_range=(0.08, 0.20)
            ),
            'MARGEM_LIQUIDA': ScoringMetric(
                name='MARGEM_LIQUIDA', weight=0.05, direction=1,  # Reduzido de 0.06 para 0.05
                description='Margem Líquida - Lucro/Receita',
                ideal_range=(0.06, 0.18)
            ),
            'MARGEM_BRUTA': ScoringMetric(
                name='MARGEM_BRUTA', weight=0.04, direction=1,
                description='Margem Bruta - Lucro Bruto/Receita',
                ideal_range=(0.20, 0.50)
            ),
            
            # === SOLVÊNCIA E ALAVANCAGEM (33% do total) ===
            'ALAVANCAGEM': ScoringMetric(
                name='ALAVANCAGEM', weight=0.09, direction=-1,  # Aumentado de 0.08 para 0.09
                description='Alavancagem Total - Passivo/Ativo',
                ideal_range=(0.30, 0.60)
            ),
            'DIVIDA_PL': ScoringMetric(
                name='DIVIDA_PL', weight=0.09, direction=-1,  # Aumentado de 0.08 para 0.09
                description='Dívida/Patrimônio Líquido',
                ideal_range=(0.50, 1.50)
            ),
            'LIQUIDEZ_CORRENTE': ScoringMetric(
                name='LIQUIDEZ_CORRENTE', weight=0.08, direction=1,  # Aumentado de 0.07 para 0.08
                description='Liquidez Corrente - Ativo Circulante/Passivo Circulante',
                ideal_range=(1.20, 3.00)
            ),
            'GIRO_ATIVO': ScoringMetric(
                name='GIRO_ATIVO', weight=0.07, direction=1,
                description='Giro do Ativo - Receita/Ativo Total',
                ideal_range=(0.30, 1.00)
            ),
            
            # === CRESCIMENTO (10% do total) ===
            'CRESC_RECEITA': ScoringMetric(
                name='CRESC_RECEITA', weight=0.05, direction=1,
                description='Crescimento da Receita (anual)',
                ideal_range=(0.05, 0.30)
            ),
            'CRESC_LUCRO': ScoringMetric(
                name='CRESC_LUCRO', weight=0.05, direction=1,
                description='Crescimento do Lucro Líquido (anual)',
                ideal_range=(0.08, 0.40)
            ),
            
            # === EFICIÊNCIA (10% do total) ===
            'SENTIMENT_MEDIO': ScoringMetric(
                name='SENTIMENT_MEDIO', weight=0.05, direction=1,
                description='Sentimento Médio de Notícias',
                ideal_range=(0.10, 0.80)
            ),
            'VOLATILIDADE': ScoringMetric(
                name='VOLATILIDADE', weight=0.05, direction=-1,
                description='Volatilidade dos Retornos (3 meses)',
                ideal_range=(0.10, 0.40)
            )
        }
        
        # Validar que soma dos pesos = 1.0
        total_weight = sum(metric.weight for metric in metrics_config.values())
        if abs(total_weight - 1.0) > 0.001:
            # Ajuste automático para garantir soma = 1.0
            adjustment_factor = 1.0 / total_weight
            for metric in metrics_config.values():
                metric.weight *= adjustment_factor
            
            logger.info(f"🔧 Pesos ajustados automaticamente para soma = 1.0")
        
        logger.info(f"✅ Sistema de scoring inicializado com {len(metrics_config)} métricas")
        logger.info(f"📊 Soma dos pesos: {sum(metric.weight for metric in metrics_config.values()):.3f}")
        
        return metrics_config
    
    def _load_custom_config(self, config_path: str) -> Dict[str, ScoringMetric]:
        """Carrega configuração personalizada de pesos"""
        try:
            with open(config_path, 'r', encoding='utf-8') as f:
                config_data = json.load(f)
            
            metrics_config = {}
            for metric_name, metric_data in config_data.items():
                metrics_config[metric_name] = ScoringMetric(**metric_data)
            
            logger.info(f"✅ Configuração personalizada carregada: {config_path}")
            return metrics_config
            
        except Exception as e:
            logger.error(f"❌ Erro ao carregar configuração: {e}")
            return self._initialize_scoring_metrics()  # Fallback para padrão
    
    def calculate_individual_scores(self, df: pd.DataFrame) -> pd.DataFrame:
        """
        Calcula scores individuais para cada métrica usando normalização avançada
        """
        scores_df = df.copy()
        
        logger.info("🎯 Calculando scores individuais...")
        
        for metric_name, metric_config in self.scoring_metrics.items():
            if metric_name not in scores_df.columns:
                logger.warning(f"⚠️ Métrica {metric_name} não encontrada no dataset")
                continue
            
            # CORREÇÃO: Verificação de tipo de dados corrigida para NumPy 2.0
            if scores_df[metric_name].dtype == object or pd.api.types.is_string_dtype(scores_df[metric_name]):
                logger.warning(f"⚠️ Métrica {metric_name} é do tipo texto - pulando")
                continue
            
            # Remover outliers extremos
            clean_series = self._remove_outliers(scores_df[metric_name])
            
            if metric_config.direction == 1:
                # MAIOR é melhor - usar função sigmoide para suavizar
                scores_df[f'score_{metric_name}'] = self._sigmoid_normalization(clean_series)
            else:
                # MENOR é melhor - inverter a normalização
                scores_df[f'score_{metric_name}'] = 1 - self._sigmoid_normalization(clean_series)
            
            # Aplicar pesos
            scores_df[f'score_{metric_name}'] *= 100  # Converter para 0-100
            scores_df[f'score_{metric_name}'] *= metric_config.weight
            
            valid_scores = scores_df[f'score_{metric_name}'].notna().sum()
            logger.info(f"  ✅ {metric_name}: {valid_scores} scores calculados (peso: {metric_config.weight*100:.1f}%)")
        
        return scores_df
    
    def _remove_outliers(self, series: pd.Series, n_std: int = 3) -> pd.Series:
        """Remove outliers usando método Z-score"""
        # CORREÇÃO: Verificação de tipo de dados atualizada para NumPy 2.0
        if series.dtype == object or pd.api.types.is_string_dtype(series) or pd.api.types.is_categorical_dtype(series):
            return series
        
        try:
            z_scores = np.abs((series - series.mean()) / series.std())
            clean_series = series.copy()
            clean_series[z_scores > n_std] = np.nan
            
            outliers_removed = (z_scores > n_std).sum()
            if outliers_removed > 0:
                logger.debug(f"  🎯 {outliers_removed} outliers removidos de {series.name}")
            
            return clean_series
        except Exception as e:
            logger.warning(f"  ⚠️ Erro ao remover outliers de {series.name}: {e}")
            return series
    
    def _sigmoid_normalization(self, series: pd.Series) -> pd.Series:
        """
        Normalização usando função sigmoide para suavizar valores extremos
        Mais robusta que rankeamento simples
        """
        # CORREÇÃO: Verificação de tipo de dados atualizada
        if series.dtype == object or pd.api.types.is_string_dtype(series) or pd.api.types.is_categorical_dtype(series):
            return series
        
        try:
            # Standardizar para média 0, std 1
            standardized = (series - series.mean()) / series.std()
            
            # Aplicar sigmoide
            sigmoid = 1 / (1 + np.exp(-standardized))
            
            return sigmoid
        except Exception as e:
            logger.warning(f"  ⚠️ Erro na normalização sigmoide de {series.name}: {e}")
            # Fallback: normalização linear simples
            return (series - series.min()) / (series.max() - series.min())
    
    def calculate_composite_score(self, scores_df: pd.DataFrame) -> pd.DataFrame:
        """
        Calcula o score composto final com validações
        """
        logger.info("⚖️ Calculando score composto...")
        
        # Identificar colunas de score
        score_columns = [col for col in scores_df.columns if col.startswith('score_')]
        
        if not score_columns:
            raise ValueError("❌ Nenhuma coluna de score encontrada")
        
        logger.info(f"📋 Colunas de score encontradas: {len(score_columns)}")
        
        # Calcular score composto
        scores_df['aurum_quality_score'] = scores_df[score_columns].sum(axis=1, skipna=True)
        
        # Normalizar para 0-100 (caso alguns pesos não tenham sido aplicados)
        max_possible_score = sum(metric.weight * 100 for metric in self.scoring_metrics.values())
        scores_df['aurum_quality_score'] = (scores_df['aurum_quality_score'] / max_possible_score) * 100
        
        # Garantir que scores estejam entre 0 e 100
        scores_df['aurum_quality_score'] = scores_df['aurum_quality_score'].clip(0, 100)
        
        # Aplicar classificações
        scores_df = self._apply_quality_classifications(scores_df)
        
        valid_scores = scores_df['aurum_quality_score'].notna().sum()
        logger.info(f"✅ Score composto calculado: {valid_scores} empresas")
        
        return scores_df
    
    def _apply_quality_classifications(self, df: pd.DataFrame) -> pd.DataFrame:
        """Aplica classificações de qualidade (A, B, C, D, E)"""
        
        # Classificação por quintis
        try:
            df['quality_quintile'] = pd.qcut(
                df['aurum_quality_score'], 
                5, 
                labels=['5º Quintil', '4º Quintil', '3º Quintil', '2º Quintil', '1º Quintil']
            )
        except ValueError as e:
            logger.warning(f"⚠️ Erro no qcut, usando cortes uniformes: {e}")
            # Fallback para cortes uniformes
            df['quality_quintile'] = pd.cut(
                df['aurum_quality_score'],
                bins=5,
                labels=['5º Quintil', '4º Quintil', '3º Quintil', '2º Quintil', '1º Quintil']
            )
        
        # Classificação por letras baseada em thresholds
        conditions = [
            df['aurum_quality_score'] >= self.quality_thresholds['A'],
            df['aurum_quality_score'] >= self.quality_thresholds['B'],
            df['aurum_quality_score'] >= self.quality_thresholds['C'],
            df['aurum_quality_score'] >= self.quality_thresholds['D'],
            df['aurum_quality_score'] >= self.quality_thresholds['E']
        ]
        
        choices = ['A', 'B', 'C', 'D', 'E']
        
        df['quality_grade'] = np.select(conditions, choices, default='E')
        
        # Classificação descritiva
        grade_descriptions = {
            'A': 'Excelente Qualidade',
            'B': 'Boa Qualidade', 
            'C': 'Qualidade Regular',
            'D': 'Qualidade Baixa',
            'E': 'Qualidade Muito Baixa'
        }
        
        df['quality_description'] = df['quality_grade'].map(grade_descriptions)
        
        return df
    
    def calculate_sector_adjusted_scores(self, df: pd.DataFrame, sector_column: str = 'setor') -> pd.DataFrame:
        """
        Calcula scores ajustados por setor (quando informação de setor disponível)
        """
        if sector_column not in df.columns:
            logger.warning("⚠️ Coluna de setor não encontrada - pulando ajuste setorial")
            return df
        
        logger.info("🏭 Aplicando ajuste setorial...")
        
        df_sector_adjusted = df.copy()
        
        # Calcular medianas por setor para cada métrica
        for metric_name in self.scoring_metrics.keys():
            if metric_name in df.columns:
                sector_medians = df.groupby(sector_column)[metric_name].median()
                
                # Ajustar scores baseado na mediana do setor
                df_sector_adjusted[f'{metric_name}_sector_adj'] = df.apply(
                    lambda row: row[metric_name] / sector_medians.get(row[sector_column], 1.0), 
                    axis=1
                )
        
        # Recalcular scores com ajuste setorial
        score_columns_adj = [col for col in df_sector_adjusted.columns if col.endswith('_sector_adj')]
        
        if score_columns_adj:
            logger.info(f"✅ Ajuste setorial aplicado para {len(score_columns_adj)} métricas")
        
        return df_sector_adjusted
    
    def validate_scoring_system(self, scores_df: pd.DataFrame) -> Dict:
        """
        Valida o sistema de scoring através de análises estatísticas
        """
        logger.info("🔍 Validando sistema de scoring...")
        
        validation_report = {
            'basic_stats': {
                'total_companies': len(scores_df),
                'companies_with_scores': scores_df['aurum_quality_score'].notna().sum(),
                'score_mean': scores_df['aurum_quality_score'].mean(),
                'score_std': scores_df['aurum_quality_score'].std(),
                'score_min': scores_df['aurum_quality_score'].min(),
                'score_max': scores_df['aurum_quality_score'].max(),
                'score_median': scores_df['aurum_quality_score'].median()
            },
            'distribution': {
                'grade_A': len(scores_df[scores_df['quality_grade'] == 'A']),
                'grade_B': len(scores_df[scores_df['quality_grade'] == 'B']),
                'grade_C': len(scores_df[scores_df['quality_grade'] == 'C']),
                'grade_D': len(scores_df[scores_df['quality_grade'] == 'D']),
                'grade_E': len(scores_df[scores_df['quality_grade'] == 'E'])
            },
            'correlation_analysis': {},
            'metric_contribution': {},
            'weight_summary': {}
        }
        
        # Análise de correlação entre scores individuais e score final
        score_columns = [col for col in scores_df.columns if col.startswith('score_')]
        
        for score_col in score_columns:
            if score_col in scores_df.columns:
                correlation = scores_df[score_col].corr(scores_df['aurum_quality_score'])
                validation_report['correlation_analysis'][score_col] = round(correlation, 4) if not pd.isna(correlation) else None
        
        # Contribuição de cada métrica
        total_actual_weight = 0
        for metric_name, metric_config in self.scoring_metrics.items():
            score_col = f'score_{metric_name}'
            if score_col in scores_df.columns:
                actual_contribution = scores_df[score_col].mean() / scores_df['aurum_quality_score'].mean() * 100
                if not pd.isna(actual_contribution):
                    validation_report['metric_contribution'][metric_name] = {
                        'weight': round(metric_config.weight * 100, 2),
                        'actual_contribution': round(actual_contribution, 2),
                        'description': metric_config.description
                    }
                    total_actual_weight += actual_contribution
        
        validation_report['weight_summary']['total_theoretical_weight'] = 100.0
        validation_report['weight_summary']['total_actual_weight'] = round(total_actual_weight, 2)
        
        logger.info("✅ Sistema de scoring validado")
        return validation_report
    
    def save_scoring_configuration(self, output_path: str = "data/aurum_scoring_config.json"):
        """Salva a configuração do sistema de scoring"""
        config_data = {}
        
        for metric_name, metric_config in self.scoring_metrics.items():
            config_data[metric_name] = {
                'name': metric_config.name,
                'weight': metric_config.weight,
                'direction': metric_config.direction,
                'description': metric_config.description,
                'min_value': metric_config.min_value,
                'max_value': metric_config.max_value,
                'ideal_range': metric_config.ideal_range
            }
        
        output_dir = Path(output_path).parent
        output_dir.mkdir(parents=True, exist_ok=True)
        
        with open(output_path, 'w', encoding='utf-8') as f:
            json.dump(config_data, f, indent=2, ensure_ascii=False)
        
        logger.info(f"💾 Configuração salva em: {output_path}")
        return output_path

# ==================== EXECUÇÃO PRINCIPAL ====================

def run_advanced_scoring_system():
    """
    Executa o sistema avançado de scoring completo
    """
    logger.info("🚀 INICIANDO SISTEMA AVANÇADO DE SCORING AURUM")
    
    try:
        # 1. INICIALIZAR SISTEMA DE SCORING
        scoring_system = AurumScoringSystem()
        
        # 2. CARREGAR DADOS DO DATAFRAME MESTRE (Unificado)
        ratios_path = "data/aurum_master_features.parquet" # <- ESTA É A MUDANÇA
        
        if not Path(ratios_path).exists():
            logger.error(f"❌ Arquivo de ratios não encontrado: {ratios_path}")
            logger.info("💡 Execute primeiro o cálculo dos ratios financeiros")
            return None
        
        ratios_df = pd.read_parquet(ratios_path)
        logger.info(f"✅ Dados de ratios carregados: {ratios_df.shape}")
        
        # 3. CALCULAR SCORES INDIVIDUAIS
        individual_scores_df = scoring_system.calculate_individual_scores(ratios_df)
        
        # 4. CALCULAR SCORE COMPOSTO
        final_scores_df = scoring_system.calculate_composite_score(individual_scores_df)
        
        # 5. VALIDAR SISTEMA
        validation_report = scoring_system.validate_scoring_system(final_scores_df)
        
        # 6. SALVAR RESULTADOS
        output_dir = Path("data/aurum_final_scores")
        output_dir.mkdir(parents=True, exist_ok=True)
        
        # Salvar scores finais
        final_output_path = output_dir / "aurum_advanced_scores.parquet"
        final_scores_df.to_parquet(final_output_path, index=False)
        
        # Salvar apenas os mais recentes
        latest_scores = final_scores_df.sort_values(['CNPJ_CIA', 'DT_FIM_EXERC']).groupby('CNPJ_CIA').last().reset_index()
        latest_output_path = output_dir / "aurum_latest_advanced_scores.parquet"
        latest_scores.to_parquet(latest_output_path, index=False)
        
        # Salvar configuração
        config_path = scoring_system.save_scoring_configuration()
        
        # 7. RELATÓRIO FINAL
        print("\n" + "="*70)
        print("🎉 SISTEMA DE SCORING AURUM - RESULTADOS FINAIS")
        print("="*70)
        
        basic_stats = validation_report['basic_stats']
        distribution = validation_report['distribution']
        
        print(f"\n📊 ESTATÍSTICAS GERAIS:")
        print(f"   • Empresas no dataset: {basic_stats['total_companies']:,}")
        print(f"   • Empresas com score:  {basic_stats['companies_with_scores']:,}")
        print(f"   • Score médio: {basic_stats['score_mean']:.2f}")
        print(f"   • Score mediano: {basic_stats['score_median']:.2f}")
        print(f"   • Melhor score: {basic_stats['score_max']:.2f}")
        print(f"   • Pior score: {basic_stats['score_min']:.2f}")
        
        print(f"\n📈 DISTRIBUIÇÃO DAS NOTAS:")
        print(f"   • Nota A (Excelente): {distribution['grade_A']:3d} empresas")
        print(f"   • Nota B (Boa):       {distribution['grade_B']:3d} empresas") 
        print(f"   • Nota C (Regular):   {distribution['grade_C']:3d} empresas")
        print(f"   • Nota D (Baixa):     {distribution['grade_D']:3d} empresas")
        print(f"   • Nota E (Muito Baixa): {distribution['grade_E']:3d} empresas")
        
        print(f"\n🥇 TOP 10 EMPRESAS POR QUALIDADE:")
        top_10 = latest_scores.nlargest(10, 'aurum_quality_score')[
            ['DENOM_CIA', 'aurum_quality_score', 'quality_grade', 'quality_description']
        ]
        
        for i, (_, row) in enumerate(top_10.iterrows(), 1):
            print(f"   {i:2d}. {row['DENOM_CIA'][:35]:35} {row['aurum_quality_score']:6.2f} ({row['quality_grade']})")
        
        print(f"\n📋 CONTRIBUIÇÃO DAS MÉTRICAS:")
        metric_contrib = validation_report['metric_contribution']
        for metric_name, contrib_info in list(metric_contrib.items())[:6]:  # Mostrar top 6
            diff = contrib_info['actual_contribution'] - contrib_info['weight']
            diff_symbol = "+" if diff > 0 else ""
            print(f"   • {metric_name:15}: {contrib_info['actual_contribution']:5.1f}% (peso: {contrib_info['weight']:.1f}%) {diff_symbol}{diff:+.1f}%")
        
        weight_summary = validation_report['weight_summary']
        print(f"\n⚖️  RESUMO DE PESOS:")
        print(f"   • Peso teórico total: {weight_summary['total_theoretical_weight']}%")
        print(f"   • Peso real total:    {weight_summary['total_actual_weight']}%")
        
        print(f"\n💾 ARQUIVOS GERADOS:")
        print(f"   • Scores completos: {final_output_path}")
        print(f"   • Scores recentes:  {latest_output_path}")
        print(f"   • Configuração:     {config_path}")
        
        return {
            'scoring_system': scoring_system,
            'final_scores': final_scores_df,
            'latest_scores': latest_scores,
            'validation_report': validation_report
        }
        
    except Exception as e:
        logger.error(f"❌ Erro no sistema de scoring: {e}")
        import traceback
        logger.error(traceback.format_exc())
        raise

# Função para análise rápida de métricas
def analyze_metric_importance():
    """Analisa a importância de cada métrica no sistema de scoring"""
    scoring_system = AurumScoringSystem()
    
    print("\n🔍 ANÁLISE DE IMPORTÂNCIA DAS MÉTRICAS")
    print("="*50)
    
    metrics_by_category = {
        'RENTABILIDADE': ['ROE', 'ROA', 'MARGEM_EBIT', 'MARGEM_LIQUIDA', 'MARGEM_BRUTA'],
        'SOLVÊNCIA': ['ALAVANCAGEM', 'DIVIDA_PL', 'LIQUIDEZ_CORRENTE', 'GIRO_ATIVO'],
        'CRESCIMENTO': ['CRESC_RECEITA', 'CRESC_LUCRO'],
        'EFICIÊNCIA': ['SENTIMENT_MEDIO', 'VOLATILIDADE']
    }
    
    total_weight = 0
    for category, metrics in metrics_by_category.items():
        category_weight = sum(
            scoring_system.scoring_metrics[metric].weight 
            for metric in metrics 
            if metric in scoring_system.scoring_metrics
        )
        total_weight += category_weight
        
        print(f"\n📊 {category}: {category_weight*100:.1f}%")
        for metric in metrics:
            if metric in scoring_system.scoring_metrics:
                metric_config = scoring_system.scoring_metrics[metric]
                print(f"   • {metric:20} {metric_config.weight*100:5.1f}% - {metric_config.description}")
    
    print(f"\n📈 SOMA TOTAL: {total_weight*100:.1f}%")

if __name__ == "__main__":
    # Executar sistema completo
    results = run_advanced_scoring_system()
    
    # Mostrar análise de importância
    analyze_metric_importance()
    
    print("\n🎯 PRÓXIMOS PASSOS SUGERIDOS:")
    print("   1. Analisar correlação entre scores e performance futura")
    print("   2. Ajustar pesos baseado em backtesting histórico") 
    print("   3. Implementar ajustes setoriais específicos")
    print("   4. Criar dashboard de monitoramento dos scores")

ERROR:__main__:❌ Arquivo de ratios não encontrado: data/aurum_master_features.parquet



🔍 ANÁLISE DE IMPORTÂNCIA DAS MÉTRICAS

📊 RENTABILIDADE: 47.0%
   • ROE                   18.0% - Return on Equity - Eficiência do capital próprio
   • ROA                   12.0% - Return on Assets - Eficiência dos ativos
   • MARGEM_EBIT            8.0% - Margem Operacional - Ebit/Receita
   • MARGEM_LIQUIDA         5.0% - Margem Líquida - Lucro/Receita
   • MARGEM_BRUTA           4.0% - Margem Bruta - Lucro Bruto/Receita

📊 SOLVÊNCIA: 33.0%
   • ALAVANCAGEM            9.0% - Alavancagem Total - Passivo/Ativo
   • DIVIDA_PL              9.0% - Dívida/Patrimônio Líquido
   • LIQUIDEZ_CORRENTE      8.0% - Liquidez Corrente - Ativo Circulante/Passivo Circulante
   • GIRO_ATIVO             7.0% - Giro do Ativo - Receita/Ativo Total

📊 CRESCIMENTO: 10.0%
   • CRESC_RECEITA          5.0% - Crescimento da Receita (anual)
   • CRESC_LUCRO            5.0% - Crescimento do Lucro Líquido (anual)

📊 EFICIÊNCIA: 10.0%
   • SENTIMENT_MEDIO        5.0% - Sentimento Médio de Notícias
   • VOLATILIDA


---

### **Documentação**: Script de Coleta de Notícias (Google News RSS)

#### **1. Objetivo**

Este script implementa parte do Pilar 2 (Qualidade da Comunicação) do projeto Aurum. Sua responsabilidade é automatizar a coleta de notícias financeiras recentes para cada empresa (ticker) listada no índice IBRX-100.

O script utiliza o *feed RSS do Google News* como fonte de dados, buscando menções a cada ticker nos últimos 30 dias. Os dados brutos coletados são a base para a futura análise de sentimento (NLP).

#### **2. Configuração (Input)**

O script depende de um único arquivo de entrada:

* `tickers_ibrx100_full.csv`: Um arquivo CSV que deve conter a lista completa de tickers do IBRX-100.
    * Formato esperado: O script lê a **primeira coluna** deste arquivo. Os tickers podem estar no formato `PETR4.SA` ou `PETR4`. A função `load_tickers_from_csv` remove automaticamente o sufixo `.SA` para otimizar a busca no Google News.

#### **3. Saída (Output)**

O script gera dois arquivos idênticos em conteúdo, localizados em `data/news/`:

1.  `raw_news_data.parquet`
2.  `raw_news_data.csv`

O schema (colunas) do DataFrame salvo é:

| Coluna | Tipo | Descrição |
| :--- | :--- | :--- |
| `ticker_query` | string | O ticker usado na busca (ex: `PETR4`). |
| `title` | string | O título da notícia. |
| `link` | string | O link original da notícia. |
| `published_date` | datetime | A data e hora da publicação (já convertida). |
| `source` | string | O nome do veículo de mídia (ex: "InfoMoney"). |
| `summary` | string | Um pequeno resumo ou *snippet* da notícia (HTML). |

---

In [ ]:
# Diretório para salvar os dados
DATA_DIR = "data"
NEWS_DIR = os.path.join(DATA_DIR, "news")
os.makedirs(NEWS_DIR, exist_ok=True)

def load_tickers_from_csv(file_path: str) -> list:
    """Carrega a lista de tickers a partir de um arquivo CSV."""
    df = pd.read_csv(file_path)
    tickers = df.iloc[:, 0].dropna().astype(str).tolist()
    # Remove o sufixo '.SA' para usar na busca de notícias
    return [t.replace('.SA', '') for t in tickers]

def fetch_news_for_ticker(ticker: str):
    """Busca notícias para um ticker específico usando o RSS do Google News."""
    raw_query = f'"{ticker}" when:30d'
    search_query = urllib.parse.quote(raw_query)
    url = f"https://news.google.com/rss/search?q={search_query}&hl=pt-BR&gl=BR&ceid=BR:pt-419"
    
    feed = feedparser.parse(url)
    
    news_items = []
    for entry in feed.entries:
        news_items.append({
            'ticker_query': ticker,
            'title': entry.title,
            'link': entry.link,
            'published_date': entry.published,
            'source': entry.source.title,
            'summary': entry.summary
        })
    return news_items


if __name__ == "__main__":
    tickers_csv_path = "tickers_ibrx100_full.csv"
    tickers = load_tickers_from_csv(tickers_csv_path)
    
    all_news = []
    
    print("Iniciando a coleta de notícias via Google News RSS...")
    for ticker in tqdm(tickers, desc="Buscando notícias"):
        try:
            news = fetch_news_for_ticker(ticker)
            if news:
                all_news.extend(news)
            time.sleep(0.5)
        except Exception as e:
            print(f"Erro ao buscar notícias para {ticker}: {e}")

    if not all_news:
        print("\nNenhuma notícia foi coletada. Verifique a conexão ou a consulta de busca. Encerrando.")
    else:
        df_news = pd.DataFrame(all_news)
        df_news.drop_duplicates(subset=['title', 'link'], inplace=True)
        df_news['published_date'] = pd.to_datetime(df_news['published_date'])
        
        # --- SALVANDO ARQUIVOS ---
        output_path_parquet = os.path.join(NEWS_DIR, "raw_news_data.parquet")
        output_path_csv = os.path.join(NEWS_DIR, "raw_news_data.csv")

        # Salva em Parquet
        df_news.to_parquet(output_path_parquet, index=False)
        
        # Salva em CSV
        df_news.to_csv(output_path_csv, index=False)
        
        print(f"\nColeta concluída. {len(df_news)} notícias únicas salvas.")
        print(f"-> {output_path_parquet}")
        print(f"-> {output_path_csv}")
        print("\nAmostra das notícias coletadas:")
        print(df_news.head())

### **Documentação:** Script de Amostragem e Análise (AurumDataSampler)

#### **1. Objetivo**

Este é um **script utilitário** de Análise Exploratória de Dados (EDA). Seu principal objetivo é ajudar o desenvolvedor a entender o ecossistema de dados do projeto Aurum, que está em constante evolução.

Ele **não coleta** novos dados. Em vez disso, ele **varre** o diretório `data/` em busca de todos os arquivos de dados `.parquet` já processados e, para cada um, executa as seguintes ações:
1.  **Descobre:** Lista todos os datasets `.parquet` disponíveis.
2.  **Amostra:** Carrega uma amostra pequena e inteligente dos dados (usando amostragem estratificada por data, se possível).
3.  **Analisa:** Gera um relatório básico de qualidade dos dados (contagem de linhas, colunas, tipos de dados, valores nulos, duplicatas e estatísticas numéricas).
4.  **Reporta:** Salva as amostras em arquivos `.csv` fáceis de visualizar e cria um relatório consolidado em `.json`.

Este script é fundamental para validar rapidamente a saída de cada etapa do pipeline (coleta de notícias, processamento de fundamentos, etc.).

#### **2. Configuração (Input)**

O script foi projetado para funcionar sem configuração manual. Ele assume a seguinte estrutura de diretório:

* **data/ (Diretório Raiz):** O script inicia sua busca aqui.
* **.parquet** (Arquivos Alvo):** Ele procurará recursivamente em **todos** os subdiretórios dentro de data/ por qualquer arquivo que termine com a extensão .parquet.

## 3. Saída (Output)

Ao executar main(), o script cria um novo diretório: data/samples/. Este diretório conterá:

1.  **Amostras em CSV (`*.sample.csv`):**
    * `raw_news_data_sample.csv`
    * `fundamentals_wide_sample.csv`
    * `aurum_quality_scores_complete_sample.csv`
    * *(...e um `.csv` para cada `.parquet` encontrado)*

2.  **Relatório Consolidado (`aurum_sampling_report.json`):**
    * Um único arquivo JSON que armazena os metadados e a análise de qualidade de todos os datasets processados. Este arquivo é ideal para monitoramento programático do "data health" (saúde dos dados) do projeto.

---

In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path
import logging
from typing import Dict, List, Optional
import json

# Configuração de logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

class AurumDataSampler:
    """
    Classe para amostragem e análise exploratória dos dados do Aurum
    """
    
    def __init__(self, base_path: str = "data"):
        self.base_path = Path(base_path)
        self.sample_size = 100  # Tamanho padrão da amostra
        
    def list_available_datasets(self) -> Dict[str, Path]:
        """
        Lista todos os datasets Parquet disponíveis
        """
        datasets = {}
        
        # Procurar recursivamente por arquivos .parquet
        for parquet_file in self.base_path.rglob("*.parquet"):
            relative_path = parquet_file.relative_to(self.base_path)
            datasets[str(relative_path)] = parquet_file
            
        return datasets
    
    def load_dataset_sample(self, dataset_path: Path, sample_size: int = None, 
                          random_state: int = 42) -> pd.DataFrame:
        """
        Carrega uma amostra representativa do dataset
        """
        if sample_size is None:
            sample_size = self.sample_size
            
        logger.info(f"📊 Carregando amostra de {dataset_path}...")
        
        try:
            # Ler o dataset completo para obter metadados
            full_df = pd.read_parquet(dataset_path)
            total_rows = len(full_df)
            
            logger.info(f"   📈 Dataset completo: {total_rows} linhas, {len(full_df.columns)} colunas")
            
            # Estratégias de amostragem baseadas no tamanho do dataset
            if total_rows <= sample_size:
                # Dataset pequeno - retornar tudo
                sample_df = full_df.copy()
                logger.info("   🔍 Dataset pequeno - retornando dados completos")
            else:
                # Dataset grande - amostragem estratificada quando possível
                sample_df = self._stratified_sampling(full_df, sample_size, random_state)
                
            return sample_df
            
        except Exception as e:
            logger.error(f"❌ Erro ao carregar {dataset_path}: {e}")
            return pd.DataFrame()
    
    def _stratified_sampling(self, df: pd.DataFrame, sample_size: int, random_state: int) -> pd.DataFrame:
        """
        Amostragem estratificada tentando manter proporções de categorias importantes
        """
        sample_df = df.copy()
        
        # Tentar estratificar por ano se a coluna existir
        year_columns = [col for col in sample_df.columns if 'ANO' in col or 'YEAR' in col or 'DATA' in col]
        
        if year_columns:
            try:
                year_col = year_columns[0]
                # Amostragem estratificada por ano
                stratified_sample = sample_df.groupby(year_col, group_keys=False).apply(
                    lambda x: x.sample(n=min(len(x), max(1, sample_size // sample_df[year_col].nunique())), 
                                      random_state=random_state)
                )
                
                # Se a amostra estratificada for muito pequena, completar com amostra aleatória
                if len(stratified_sample) < sample_size:
                    remaining = sample_size - len(stratified_sample)
                    additional_sample = sample_df.drop(stratified_sample.index).sample(
                        n=remaining, random_state=random_state)
                    stratified_sample = pd.concat([stratified_sample, additional_sample])
                
                logger.info(f"   🎯 Amostragem estratificada por {year_col}: {len(stratified_sample)} linhas")
                return stratified_sample
                
            except Exception as e:
                logger.warning(f"   ⚠️ Amostragem estratificada falhou: {e}")
        
        # Fallback: amostra aleatória simples
        simple_sample = sample_df.sample(n=min(sample_size, len(sample_df)), random_state=random_state)
        logger.info(f"   🎲 Amostra aleatória simples: {len(simple_sample)} linhas")
        return simple_sample
    
    def generate_basic_report(self, dataset_path: Path, sample_size: int = 50) -> Dict:
        """
        Gera relatório básico do dataset (sem dados complexos para JSON)
        """
        # Carregar amostra
        sample_df = self.load_dataset_sample(dataset_path, sample_size)
        
        if sample_df.empty:
            return {
                'dataset_name': dataset_path.name,
                'dataset_path': str(dataset_path),
                'error': 'Falha ao carregar dataset'
            }
        
        # Estatísticas básicas
        basic_stats = {
            'total_rows_original': len(pd.read_parquet(dataset_path)),
            'sample_size': len(sample_df),
            'columns_count': len(sample_df.columns),
            'memory_usage_mb': round(sample_df.memory_usage(deep=True).sum() / 1024**2, 2),
            'columns_list': list(sample_df.columns),
            'dtypes': {col: str(dtype) for col, dtype in sample_df.dtypes.items()}
        }
        
        # Informações de data
        date_info = self._get_date_info(sample_df)
        
        # Qualidade básica dos dados
        quality_info = self._get_basic_quality_info(sample_df)
        
        report = {
            'dataset_name': dataset_path.name,
            'dataset_path': str(dataset_path),
            'basic_stats': basic_stats,
            'date_info': date_info,
            'quality_info': quality_info
        }
        
        return report
    
    def _get_date_info(self, df: pd.DataFrame) -> Dict:
        """Extrai informações de data de forma segura"""
        date_info = {}
        date_columns = df.select_dtypes(include=['datetime64']).columns
        
        for col in date_columns:
            try:
                if col in df.columns and not df[col].isna().all():
                    date_info[col] = {
                        'min': str(df[col].min()),
                        'max': str(df[col].max()),
                        'null_count': int(df[col].isna().sum()),
                        'null_percentage': round((df[col].isna().sum() / len(df)) * 100, 2)
                    }
            except Exception as e:
                date_info[col] = {'error': f'Erro ao processar: {str(e)}'}
        
        return date_info
    
    def _get_basic_quality_info(self, df: pd.DataFrame) -> Dict:
        """Analisa qualidade básica dos dados"""
        quality_info = {
            'total_rows': len(df),
            'complete_cases': len(df.dropna()),
            'duplicate_rows': int(df.duplicated().sum()),
            'columns_quality': {}
        }
        
        for column in df.columns:
            try:
                col_data = df[column]
                col_info = {
                    'dtype': str(col_data.dtype),
                    'null_count': int(col_data.isna().sum()),
                    'null_percentage': round((col_data.isna().sum() / len(df)) * 100, 2),
                    'unique_count': int(col_data.nunique())
                }
                
                # Estatísticas para colunas numéricas
                if pd.api.types.is_numeric_dtype(col_data):
                    col_info.update({
                        'mean': float(col_data.mean()) if not col_data.isna().all() else None,
                        'std': float(col_data.std()) if not col_data.isna().all() else None,
                        'min': float(col_data.min()) if not col_data.isna().all() else None,
                        'max': float(col_data.max()) if not col_data.isna().all() else None
                    })
                
                quality_info['columns_quality'][column] = col_info
                
            except Exception as e:
                quality_info['columns_quality'][column] = {'error': f'Erro na análise: {str(e)}'}
        
        return quality_info
    
    def create_simple_sampling_report(self, output_dir: str = "data/samples") -> Dict:
        """
        Cria relatório simplificado de amostragem para todos os datasets
        """
        output_path = Path(output_dir)
        output_path.mkdir(parents=True, exist_ok=True)
        
        datasets = self.list_available_datasets()
        reports = {}
        
        logger.info("🔍 INICIANDO AMOSTRAGEM DOS DATASETS AURUM")
        logger.info(f"📁 Encontrados {len(datasets)} datasets:")
        
        for name, path in datasets.items():
            logger.info(f"   📊 {name}")
        
        print("\n" + "="*60)
        print("🎯 RELATÓRIO DE AMOSTRAGEM - AURUM DATA SAMPLES")
        print("="*60)
        
        for dataset_name, dataset_path in datasets.items():
            print(f"\n📁 DATASET: {dataset_name}")
            print("-" * 40)
            
            # Gerar relatório básico
            report = self.generate_basic_report(dataset_path)
            reports[dataset_name] = report
            
            # Carregar amostra para salvar como CSV
            sample_df = self.load_dataset_sample(dataset_path, 50)
            
            if not sample_df.empty:
                # Salvar amostra como CSV
                sample_csv_path = output_path / f"{dataset_path.stem}_sample.csv"
                sample_df.to_csv(sample_csv_path, index=False, encoding='utf-8')
                
                # Mostrar resumo no console
                basic_stats = report['basic_stats']
                print(f"📈 Estatísticas:")
                print(f"   • Linhas totais: {basic_stats['total_rows_original']:,}")
                print(f"   • Amostra: {basic_stats['sample_size']} linhas")
                print(f"   • Colunas: {basic_stats['columns_count']}")
                print(f"   • Uso de memória: {basic_stats['memory_usage_mb']:.2f} MB")
                
                # Mostrar primeiras linhas
                print(f"\n👀 Primeiras 3 linhas da amostra:")
                print(sample_df.head(3).to_string(index=False))
                
                print(f"\n💾 Amostra salva em: {sample_csv_path}")
            else:
                print("❌ Não foi possível carregar amostra deste dataset")
        
        # Salvar relatório consolidado (agora seguro para JSON)
        report_path = output_path / "aurum_sampling_report.json"
        
        try:
            with open(report_path, 'w', encoding='utf-8') as f:
                json.dump(reports, f, indent=2, ensure_ascii=False, default=str)
            logger.info(f"📋 Relatório consolidado salvo em: {report_path}")
        except Exception as e:
            logger.error(f"❌ Erro ao salvar relatório JSON: {e}")
        
        return reports

    def quick_preview(self, dataset_pattern: str = None, sample_size: int = 10):
        """
        Visualização rápida dos datasets
        """
        datasets = self.list_available_datasets()
        
        print("🚀 VISUALIZAÇÃO RÁPIDA DOS DATASETS AURUM")
        print("=" * 50)
        
        for name, path in datasets.items():
            if dataset_pattern and dataset_pattern.lower() not in name.lower():
                continue
                
            print(f"\n📊 {name}")
            print("-" * 40)
            
            sample_df = self.load_dataset_sample(path, sample_size)
            if not sample_df.empty:
                print(f"📋 Shape: {sample_df.shape}")
                print(f"🎯 Amostra de {len(sample_df)} linhas:")
                print(sample_df.to_string(index=False))
            else:
                print("❌ Não foi possível carregar amostra")
            
            print("\n" + "=" * 50)

# Funções de uso rápido
def quick_sample(dataset_pattern: str, sample_size: int = 15) -> pd.DataFrame:
    """
    Função rápida para obter amostra de datasets que correspondam ao padrão
    """
    sampler = AurumDataSampler()
    datasets = sampler.list_available_datasets()
    
    matching_datasets = []
    for name, path in datasets.items():
        if dataset_pattern.lower() in name.lower():
            matching_datasets.append((name, path))
    
    if not matching_datasets:
        available = "\n".join(datasets.keys())
        print(f"❌ Nenhum dataset encontrado com '{dataset_pattern}'.")
        print(f"📁 Datasets disponíveis:\n{available}")
        return pd.DataFrame()
    
    if len(matching_datasets) == 1:
        name, path = matching_datasets[0]
        sample = sampler.load_dataset_sample(path, sample_size)
        print(f"🎯 Amostra de {name} ({len(sample)} linhas):")
        print(sample.to_string(index=False))
        return sample
    else:
        print(f"🔍 Múltiplos datasets encontrados com '{dataset_pattern}':")
        for i, (name, path) in enumerate(matching_datasets, 1):
            print(f"   {i}. {name}")
        
        choice = input("👉 Escolha o número do dataset (ou Enter para o primeiro): ").strip()
        try:
            choice_idx = int(choice) - 1 if choice else 0
            name, path = matching_datasets[choice_idx]
            sample = sampler.load_dataset_sample(path, sample_size)
            print(f"🎯 Amostra de {name} ({len(sample)} linhas):")
            print(sample.to_string(index=False))
            return sample
        except (ValueError, IndexError):
            print("❌ Escolha inválida.")
            return pd.DataFrame()

def list_datasets():
    """Lista todos os datasets disponíveis"""
    sampler = AurumDataSampler()
    datasets = sampler.list_available_datasets()
    
    print("📁 DATASETS DISPONÍVEIS NO AURUM:")
    print("=" * 50)
    
    for i, (name, path) in enumerate(datasets.items(), 1):
        try:
            # Tentar obter informações básicas
            df_sample = pd.read_parquet(path, nrows=1)
            print(f"{i:2d}. {name}")
            print(f"    📊 Colunas: {len(df_sample.columns)}, Estilo: {df_sample.shape[1]}xN")
            print(f"    📍 {path}")
        except:
            print(f"{i:2d}. {name} (❌ erro ao carregar)")
    
    return datasets

# Execução principal
def main():
    """
    Executa amostragem completa de todos os datasets
    """
    sampler = AurumDataSampler()
    reports = sampler.create_simple_sampling_report()
    
    print("\n" + "="*60)
    print("✅ AMOSTRAGEM CONCLUÍDA!")
    print("="*60)
    
    # Resumo final
    total_datasets = len(reports)
    successful_samples = sum(1 for report in reports.values() if 'error' not in report)
    
    print(f"📊 Resumo Final:")
    print(f"   • Datasets processados: {total_datasets}")
    print(f"   • Amostras geradas com sucesso: {successful_samples}")
    print(f"   • Local das amostras: data/samples/")
    
    print(f"\n🎯 COMANDOS RÁPIDOS:")
    print(f"   • list_datasets() - Lista todos os datasets")
    print(f"   • quick_sample('quality') - Amostra de datasets de qualidade")
    print(f"   • quick_sample('news') - Amostra de datasets de notícias")
    print(f"   • quick_sample('fundamentals') - Amostra de dados fundamentalistas")

if __name__ == "__main__":
    main()

# Exemplos de uso após execução
print("\n" + "="*60)
print("🚀 EXEMPLOS DE USO RÁPIDO - EXECUTE APÓS O SCRIPT:")
print("="*60)
print("""
# Listar todos os datasets
list_datasets()

# Amostra rápida de datasets de qualidade
quick_sample('quality', 10)

# Amostra de dados fundamentalistas  
quick_sample('fundamental', 15)

# Amostra de notícias
quick_sample('news', 8)

# Visualização rápida de tudo
sampler = AurumDataSampler()
sampler.quick_preview()
""")


🎯 RELATÓRIO DE AMOSTRAGEM - AURUM DATA SAMPLES

📁 DATASET: aurum_master_features.parquet
----------------------------------------
📈 Estatísticas:
   • Linhas totais: 19,581
   • Amostra: 50 linhas
   • Colunas: 57
   • Uso de memória: 0.03 MB

👀 Primeiras 3 linhas da amostra:
      date   ticker  Adj Close           CNPJ_CIA                                DENOM_CIA  Custo dos Bens e/ou Serviços Vendidos         EBIT          EBT   Lucro Bruto  Lucro Líquido Consolidado  Receita Líquida  Ativo Circulante  Ativo Não Circulante  Ativo Total  Caixa e Equivalentes  Dívida Curto Prazo  Dívida Longo Prazo  Passivo Circulante  Passivo Não Circulante  Passivo Total  Patrimônio Líquido Consolidado  Receita Líquida_ttm  Custo dos Bens e/ou Serviços Vendidos_ttm  Lucro Bruto_ttm     EBIT_ttm      EBT_ttm  Lucro Líquido Consolidado_ttm  Dívida Bruta  Capital Investido  Dívida Líquida       ROE       ROA      ROIC  MARGEM_EBIT  MARGEM_LIQUIDA  MARGEM_BRUTA  ALAVANCAGEM  DIVIDA_PL  DIVIDA_LIQ_EBIT  

: 